In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.compose import ColumnTransformer
import statsmodels.api as sma
from sklearn.model_selection import cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn import tree
from scipy.stats import randint
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [2]:
df1=pd.read_csv("df_trim.csv")
df1.head()

,product_id,time_id,customer_id,promotion_id,store_id,store_sales,store_cost,unit_sales,product_class_id,brand_name,...,houseowner,num_cars_owned,fullname,the_date,the_day,the_year,the_month,day_of_month,day_since_epoch,promotion_period
0,451,677,7366,207,3,7.12,2.2784,4,55,Red Wing,...,Y,4,Ann Smith,11/7/1997,Friday,1997,11,7,34279,4
1,1293,677,7366,207,3,5.76,1.7856,4,14,Booker,...,Y,4,Ann Smith,11/7/1997,Friday,1997,11,7,34279,4
2,738,677,7366,207,3,5.55,2.0535,3,39,Consolidated,...,Y,4,Ann Smith,11/7/1997,Friday,1997,11,7,34279,4
3,1426,677,8947,207,3,7.48,2.2440,2,61,Hermanos,...,Y,2,Robert Bell,11/7/1997,Friday,1997,11,7,34279,4
4,338,677,8947,207,3,2.28,1.0488,4,58,Better,...,Y,2,Robert Bell,11/7/1997,Friday,1997,11,7,34279,4


In [3]:
#Impute null values with median for specified columns
columns_to_impute = ['store_sqft', 'grocery_sqft', 'frozen_sqft', 'meat_sqft']
imputer = SimpleImputer(strategy='median')
df1[columns_to_impute] = imputer.fit_transform(df1[columns_to_impute])

In [4]:
# Check for remaining null values
df_null = df1.isnull().sum()
df_null

product_id          0
time_id             0
customer_id         0
promotion_id        0
store_id            0
                   ..
the_year            0
the_month           0
day_of_month        0
day_since_epoch     0
promotion_period    0
Length: 94, dtype: int64

In [5]:
# Remove specified columns
columns_to_remove = ["lname", "fname", "mi", "address1", "address2"]
df1 = df1.drop(columns=columns_to_remove)

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70751 entries, 0 to 70750
Data columns (total 89 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_id             70751 non-null  int64  
 1   time_id                70751 non-null  int64  
 2   customer_id            70751 non-null  int64  
 3   promotion_id           70751 non-null  int64  
 4   store_id               70751 non-null  int64  
 5   store_sales            70751 non-null  float64
 6   store_cost             70751 non-null  float64
 7   unit_sales             70751 non-null  int64  
 8   product_class_id       70751 non-null  int64  
 9   brand_name             70751 non-null  object 
 10  product_name           70751 non-null  object 
 11  SKU                    70751 non-null  int64  
 12  SRP                    70751 non-null  float64
 13  gross_weight           70751 non-null  float64
 14  net_weight             70751 non-null  float64
 15  re

# keeping 43 columns where names are removed

In [7]:
#Taking columns that are needed
df_new=df1[['recyclable_package', 'low_fat', 'department', 'family', 'promotion_name', 'media_type', 'sales_district', 'sales_region', 'store_type', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist', 'state_province', 'country', 'marital_status', 'yearly_income', 'gender', 'education','member_card', 'occupation', 'houseowner','store_sales', 'store_cost', 'unit_sales', 'SRP', 'gross_weight', 'net_weight', 'units_per_case', 'cases_per_pallet', 'shelf_width', 'shelf_height', 'shelf_depth', 'cost', 'store_sqft', 'grocery_sqft', 'frozen_sqft', 'meat_sqft', 'total_children', 'num_children_at_home', 'num_cars_owned', 'promotion_period']]

In [8]:
df_new['department'].unique()

array(['Household', 'Dairy', 'Health and Hygiene', 'Produce',
       'Canned Foods', 'Alcoholic Beverages', 'Frozen Foods',
       'Baking Goods', 'Beverages', 'Deli', 'Checkout', 'Snack Foods',
       'Baked Goods', 'Periodicals', 'Snacks', 'Starchy Foods', 'Eggs',
       'Breakfast Foods', 'Canned Products', 'Seafood', 'Carousel',
       'Meat'], dtype=object)

In [9]:
df_new['department'].nunique()

22

In [ ]:
'Household', 'Dairy', 'Health and Hygiene', 'Produce',
       'Canned Foods', 'Alcoholic Beverages', 'Frozen Foods',
       'Baking Goods', 'Beverages', 'Deli', 'Checkout', 'Snack Foods',
       'Baked Goods', 'Periodicals', 'Snacks', 'Starchy Foods', 'Eggs',
       'Breakfast Foods', 'Canned Products', 'Seafood', 'Carousel',
       'Meat'

In [20]:
Household, Non_Veg- ['Eggs','Seafood','Meat'],Health and Hygiene,Produce,Baking-['Baking Goods','Baked Goods'],
Ready_to_eat-['Snack Foods','Snacks','Frozen Foods','Canned Foods','Canned Products','Breakfast Foods'],'Starchy Foods',
Dairy,Deli,Checkout,Beverages-['Alcoholic Beverages','Beverages'],Carousel,Periodicals

NameError: name 'Household' is not defined

In [11]:
def func_dept(x):
    if x in ['Eggs','Seafood','Meat']:
        return "Non_Veg"
    elif x in ['Baking Goods','Baked Goods']:
        return "Baking"
    elif x in ['Snack Foods','Snacks','Frozen Foods','Canned Foods','Canned Products','Breakfast Foods']:
        return "Ready_to_eat"
    elif x in ['Alcoholic Beverages','Beverages']:
        return "Beverages"
    else:
        return x

In [12]:
df_new["department"] = df_new["department"].apply(func_dept)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\506778296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["department"] = df_new["department"].apply(func_dept)


In [13]:
df_new['department'].unique()

array(['Household', 'Dairy', 'Health and Hygiene', 'Produce',
       'Ready_to_eat', 'Beverages', 'Baking', 'Deli', 'Checkout',
       'Periodicals', 'Starchy Foods', 'Non_Veg', 'Carousel'],
      dtype=object)

In [14]:
df_new['department'].nunique()

13

In [15]:
df_new["family"].unique()

array(['Non-Consumable', 'Food', 'Drink'], dtype=object)

In [16]:
df_new['promotion_name'].unique()

array(['Super Duper Savers', 'You Save Days', 'Unbeatable Price Savers',
       'Big Time Savings', 'Price Winners', 'Sales Galore',
       'Double Down Sale', 'Savings Galore', 'Mystery Sale',
       'Three for One', 'Dollar Cutters', 'Tip Top Savings',
       'One Day Sale', 'Price Savers', 'Super Wallet Savers',
       'Double Your Savings', 'Bye Bye Baby', 'Sale Winners',
       'Weekend Markdown', 'Super Savers', 'Big Time Discounts',
       'Coupon Spectacular', 'Two for One', 'Saving Days', 'Save-It Sale',
       'Go For It', 'Dollar Days', 'Best Savings', 'Price Slashers',
       'Two Day Sale', 'Sales Days', 'Green Light Days', 'Free For All',
       'Big Promo', 'Shelf Emptiers', 'Price Destroyers',
       'Shelf Clearing Days', 'Wallet Savers', 'Fantastic Discounts',
       'Money Savers', 'Bag Stuffers', 'Price Cutters',
       'Green Light Special', 'Price Smashers', 'I Cant Believe It Sale',
       'High Roller Savings', 'Dimes Off', 'Cash Register Lottery',
       'Pick 

In [17]:
df_new['promotion_name'].nunique()

49

In [ ]:
Discount_day=['You Save Days', 'One Day Sale', 'Weekend Markdown',  'Saving Days',  'Dollar Days', 
       'Two Day Sale', 'Sales Days', 'Green Light Days',
       'Shelf Clearing Days'],
free_item=['Three for One', 'Two for One',
       'Go For It','Free For All','Shelf Emptiers', 'Bag Stuffers'],
Sale_discount=['Super Duper Savers', 'Unbeatable Price Savers',
       'Big Time Savings', 'Price Winners', 'Sales Galore',
       'Double Down Sale', 'Savings Galore', 'Mystery Sale',
       'Dollar Cutters', 'Tip Top Savings',
        'Price Savers', 'Super Wallet Savers',
       'Double Your Savings', 'Bye Bye Baby', 'Sale Winners',
        'Super Savers', 'Big Time Discounts',
       'Coupon Spectacular',  'Save-It Sale',
        'Best Savings', 'Price Slashers',
       'Price Destroyers','Wallet Savers', 'Fantastic Discounts',
       'Money Savers', 'Price Cutters',
       'Green Light Special', 'Price Smashers', 'I Cant Believe It Sale',
       'High Roller Savings', 'Dimes Off', 'Pick Your Savings']
Register_promotion=['Big Promo','Cash Register Lottery']


In [23]:
def func_prom_name(x):
    if x in ['You Save Days', 'One Day Sale', 'Weekend Markdown',  'Saving Days',  'Dollar Days', 
       'Two Day Sale', 'Sales Days', 'Green Light Days',
       'Shelf Clearing Days']:
        return "Discount_day"
    elif x in ['Three for One', 'Two for One',
       'Go For It','Free For All','Shelf Emptiers', 'Bag Stuffers']:
        return "free_item"
    elif x in ['Super Duper Savers', 'Unbeatable Price Savers',
       'Big Time Savings', 'Price Winners', 'Sales Galore',
       'Double Down Sale', 'Savings Galore', 'Mystery Sale',
       'Dollar Cutters', 'Tip Top Savings',
        'Price Savers', 'Super Wallet Savers',
       'Double Your Savings', 'Bye Bye Baby', 'Sale Winners',
        'Super Savers', 'Big Time Discounts',
       'Coupon Spectacular',  'Save-It Sale',
        'Best Savings', 'Price Slashers',
       'Price Destroyers','Wallet Savers', 'Fantastic Discounts',
       'Money Savers', 'Price Cutters',
       'Green Light Special', 'Price Smashers', 'I Cant Believe It Sale',
       'High Roller Savings', 'Dimes Off', 'Pick Your Savings']:
        return "Sale_discount"
    else:
        return "Register_promotion"
   

In [24]:
df_new['promotion_name']=df_new['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\2408714610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['promotion_name']=df_new['promotion_name'].apply(func_prom_name)


In [25]:
df_new['promotion_name'].unique()

array(['Sale_discount', 'Discount_day', 'free_item', 'Register_promotion'],
      dtype=object)

In [28]:
df_new['media_type'].unique()

array(['Cash Register Handout', 'Bulk Mail', 'Daily Paper',
       'Street Handout', 'Daily Paper, Radio', 'Product Attachment',
       'Sunday Paper, Radio', 'Radio', 'In-Store Coupon', 'Sunday Paper',
       'TV', 'Daily Paper, Radio, TV', 'Sunday Paper, Radio, TV'],
      dtype=object)

In [27]:
df_new['media_type'].nunique()

13

In [29]:
df_new['sales_district'].unique()

array(['Bremerton', 'Los Angeles', 'Bellingham', 'San Diego', 'Spokane',
       'Guadalajara', 'Marida', 'Mexico City', 'San Francisco', 'Seattle',
       'Tacoma', 'Portland', 'Walla Walla', 'Salem', 'Camacho', 'Hidalgo',
       'Yakima', 'Vancouver', 'Victoria', 'Acapulco', 'Orizaba'],
      dtype=object)

* drop sales_district and keep sales_region

In [39]:
df_new.drop(columns=["sales_district"],axis=1,inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\2880156587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.drop(columns=["sales_district"],axis=1,inplace=True)


In [31]:
df_new["sales_region"].unique()

array(['North West', 'South West', 'Mexico West', 'Mexico South',
       'Mexico Central', 'Central West', 'Canada West'], dtype=object)

In [33]:
df_new["store_type"].unique()

array(['Supermarket', 'Gourmet Supermarket', 'Small Grocery',
       'Deluxe Supermarket', 'Mid-Size Grocery'], dtype=object)

In [34]:
df_new["state_province"].unique()

array(['WA', 'CA', 'Jalisco', 'Yucatan', 'DF', 'Mexico', 'OR',
       'Zacatecas', 'BC', 'Guerrero', 'Sinaloa', 'Veracruz'], dtype=object)

In [35]:
df_new["country"].unique()

array(['USA', 'Mexico', 'Canada'], dtype=object)

In [36]:
df_new["marital_status"].unique()

array(['M', 'S'], dtype=object)

In [37]:
df_new['yearly_income'].unique()

array(['$30K - $50K', '$50K - $70K', '$150K +', '$10K - $30K',
       '$110K - $130K', '$70K - $90K', '$90K - $110K', '$130K - $150K'],
      dtype=object)

In [40]:
le=LabelEncoder()
df_new_num=df_new.select_dtypes(include=np.number)
df_new_cat=df_new.select_dtypes(exclude=np.number)

In [41]:
df_new_num

,recyclable_package,low_fat,coffee_bar,video_store,salad_bar,prepared_food,florist,store_sales,store_cost,unit_sales,...,shelf_depth,cost,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,total_children,num_children_at_home,num_cars_owned,promotion_period
0,1,0,0,0,1,1,0,7.12,2.2784,4,...,19.90,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
1,0,1,0,0,1,1,0,5.76,1.7856,4,...,11.00,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
2,1,0,0,0,1,1,0,5.55,2.0535,3,...,5.17,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
3,0,1,0,0,1,1,0,7.48,2.2440,2,...,10.40,12950,39696.0,24390.0,9184.0,6122.0,2,0,2,4
4,1,0,0,0,1,1,0,2.28,1.0488,4,...,8.98,12950,39696.0,24390.0,9184.0,6122.0,2,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,0,1,1,0,0,0,1,7.68,3.5328,3,...,20.70,13006,34452.0,27463.0,4193.0,2795.0,4,0,3,3
70747,1,0,1,0,0,0,1,2.85,0.8835,5,...,4.14,13006,34452.0,27463.0,4193.0,2795.0,4,0,3,3
70748,0,0,1,0,0,0,1,4.29,1.4586,3,...,21.10,13006,34452.0,27463.0,4193.0,2795.0,1,0,2,3
70749,1,1,1,0,0,0,1,5.18,2.0720,2,...,16.10,13006,34452.0,27463.0,4193.0,2795.0,1,0,2,3


In [42]:
df_new_cat

,department,family,promotion_name,media_type,sales_region,store_type,state_province,country,marital_status,yearly_income,gender,education,member_card,occupation,houseowner
0,Household,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
1,Dairy,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
2,Health and Hygiene,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
3,Produce,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,S,$50K - $70K,M,High School Degree,Normal,Skilled Manual,Y
4,Ready_to_eat,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,S,$50K - $70K,M,High School Degree,Normal,Skilled Manual,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,Canada,S,$110K - $130K,M,High School Degree,Silver,Management,Y
70747,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,Canada,S,$110K - $130K,M,High School Degree,Silver,Management,Y
70748,Household,Non-Consumable,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,Canada,S,$90K - $110K,M,Bachelors Degree,Golden,Professional,Y
70749,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,Canada,S,$90K - $110K,M,Bachelors Degree,Golden,Professional,Y


In [43]:
for col in df_new_cat.columns:
    df_new_cat[col] = le.fit_transform(df_new_cat[col])

In [44]:
df_new_cat

,department,family,promotion_name,media_type,sales_region,store_type,state_province,country,marital_status,yearly_income,gender,education,member_card,occupation,houseowner
0,7,2,2,1,5,4,9,2,0,4,0,2,0,4,1
1,4,1,2,1,5,4,9,2,0,4,0,2,0,4,1
2,6,2,2,1,5,4,9,2,0,4,0,2,0,4,1
3,10,1,2,1,5,4,9,2,1,5,1,2,2,4,1
4,11,1,2,1,5,4,9,2,1,5,1,2,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,10,1,2,5,0,2,0,0,1,1,1,2,3,1,1
70747,10,1,2,5,0,2,0,0,1,1,1,2,3,1,1
70748,7,2,2,5,0,2,0,0,1,7,1,0,1,3,1
70749,10,1,2,5,0,2,0,0,1,7,1,0,1,3,1


In [45]:
df_new=pd.concat([df_new_cat,df_new_num],axis=1)

In [82]:
df_new

,department,family,promotion_name,media_type,sales_district,sales_region,store_type,state_province,country,marital_status,...,shelf_depth,cost,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,total_children,num_children_at_home,num_cars_owned,promotion_period
0,14,2,38,1,2,5,4,9,2,0,...,19.90,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
1,9,1,38,1,2,5,4,9,2,0,...,11.00,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
2,13,2,38,1,2,5,4,9,2,0,...,5.17,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
3,17,1,38,1,2,5,4,9,2,1,...,10.40,12950,39696.0,24390.0,9184.0,6122.0,2,0,2,4
4,5,1,38,1,2,5,4,9,2,1,...,8.98,12950,39696.0,24390.0,9184.0,6122.0,2,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,17,1,35,5,18,0,2,0,0,1,...,20.70,13006,34452.0,27463.0,4193.0,2795.0,4,0,3,3
70747,17,1,35,5,18,0,2,0,0,1,...,4.14,13006,34452.0,27463.0,4193.0,2795.0,4,0,3,3
70748,14,2,35,5,18,0,2,0,0,1,...,21.10,13006,34452.0,27463.0,4193.0,2795.0,1,0,2,3
70749,17,1,35,5,18,0,2,0,0,1,...,16.10,13006,34452.0,27463.0,4193.0,2795.0,1,0,2,3


In [166]:
#Setting target variable
X = df_new.drop(columns=['cost'])
y = df_new['cost']

In [167]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# linear regression

In [48]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [49]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     59.33
Date:                Sun, 25 Feb 2024   Prob (F-statistic):               0.00
Time:                        21:06:48   Log-Likelihood:            -5.3172e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56558   BIC:                         1.064e+06
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  1.61e+04    290.127     55.491      0.000    1.55e+04    1.67e+04
department              -0.4101      3.174     -0.129      0.897      -6.631       5.811
family                 -24.7200     24.396     -1.013      0.311     -72.537      23.097
promotion_name         -83.8301     12.958     -6.469      0.000    -109.229     -58.432
media_type             -21.1174      3.609     -5.851      0.000     -28.191     -14.044
sales_region          -231.0705     24.114     -9.583      0.000    -278.334    -183.807
store_type            -504.5498     27.920    -18.071      0.000    -559.273    -449.827
state_province         -51.2615      4.730    -10.838      0.000     -60.532     -41.991
country                 67.0256     65.013      1.031      0.303     -60.399     194.451
marital_status          12.2285     33.984      0.360      0.719     -54.380      78.837
yearly_income           -6.7911      7.929     -0.857      0.392     -22.331       8.749
gender                 -33.6276     24.603     -1.367      0.172     -81.849      14.594
education              -10.0941      9.836     -1.026      0.305     -29.373       9.184
member_card              3.7096     13.611      0.273      0.785     -22.969      30.388
occupation               2.1919     11.673      0.188      0.851     -20.687      25.071
houseowner              41.4125     26.170      1.582      0.114      -9.880      92.705
recyclable_package       0.9042     24.796      0.036      0.971     -47.696      49.504
low_fat                  1.5499     26.365      0.059      0.953     -50.126      53.226
coffee_bar            -607.6298     57.742    -10.523      0.000    -720.805    -494.455
video_store          -1255.7989     54.280    -23.136      0.000   -1362.188   -1149.410
salad_bar             1490.0604    150.661      9.890      0.000    1194.765    1785.356
prepared_food        -1553.3279    147.779    -10.511      0.000   -1842.975   -1263.681
florist              -1163.3499     70.010    -16.617      0.000   -1300.570   -1026.130
store_sales             -7.7654     19.329     -0.402      0.688     -45.650      30.119
store_cost              34.6685     28.125      1.233      0.218     -20.457      89.794
unit_sales             -15.1066     36.472     -0.414      0.679     -86.591      56.378
SRP                    -10.3408     50.303     -0.206      0.837    -108.935      88.253
gross_weight             7.2971     17.840      0.409      0.683     -27.670      42.264
net_weight              -8.3886     17.622     -0.476      0.634     -42.928      26.151
units_per_case           0.6597      1.197      0.551      0.582      -1.686       3.006
cases_per_pallet         1.2603      4.959      0.254      0.799      -8.460      10.981
shelf_width             -1.6836      2.136     -0.788    

In [50]:

significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const                0.000000e+00
promotion_name       9.933790e-11
media_type           4.906777e-09
sales_region         9.837129e-22
store_type           8.583020e-73
state_province       2.414096e-27
coffee_bar           7.141970e-26
video_store         7.186353e-118
salad_bar            4.794213e-23
prepared_food        8.106320e-26
florist              7.369514e-62
store_sqft           6.923273e-08
grocery_sqft         4.091877e-06
frozen_sqft          1.362299e-57
meat_sqft            1.591021e-57
total_children       1.079944e-02
num_cars_owned       9.436891e-05
promotion_period     5.188594e-71
dtype: float64


In [51]:
sig_pvalues_features=significant_pvalues.index

In [52]:
sig_pvalues_features.drop("const")

Index(['promotion_name', 'media_type', 'sales_region', 'store_type',
       'state_province', 'coffee_bar', 'video_store', 'salad_bar',
       'prepared_food', 'florist', 'store_sqft', 'grocery_sqft', 'frozen_sqft',
       'meat_sqft', 'total_children', 'num_cars_owned', 'promotion_period'],
      dtype='object')

In [53]:
len(significant_pvalues)

18

In [54]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [55]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8458833.342598304
Mean squared error for test data: 8465146.346413033


In [56]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10027307.93933301  9653380.67091613  8945006.71546205]


In [57]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8945006.715462051 10027307.939333014 9541898.441903733


In [58]:
lr=LinearRegression()
model_sfs=sfs(estimator=lr,k_features="best",forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('family',
 'promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'state_province',
 'marital_status',
 'gender',
 'houseowner',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'prepared_food',
 'florist',
 'store_cost',
 'unit_sales',
 'store_sqft',
 'grocery_sqft',
 'frozen_sqft',
 'meat_sqft',
 'total_children',
 'num_cars_owned',
 'promotion_period')

In [59]:
len(model_sfs.k_feature_names_)

23

In [168]:
lr=LinearRegression()
model_sfs=sfs(estimator=lr,k_features=30,forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('family',
 'promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'state_province',
 'marital_status',
 'yearly_income',
 'gender',
 'education',
 'occupation',
 'houseowner',
 'low_fat',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'prepared_food',
 'florist',
 'store_sales',
 'store_cost',
 'unit_sales',
 'gross_weight',
 'net_weight',
 'store_sqft',
 'grocery_sqft',
 'frozen_sqft',
 'meat_sqft',
 'total_children',
 'num_cars_owned',
 'promotion_period')

In [169]:
len(model_sfs.k_feature_names_)

30

# decision tree regressor
    

In [60]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [61]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False).head(30)

,feature,importance
3,media_type,0.302020
40,promotion_period,0.163595
2,promotion_name,0.144695
35,frozen_sqft,0.065093
6,state_province,0.057220
33,store_sqft,0.050837
34,grocery_sqft,0.050440
5,store_type,0.033845
21,florist,0.020159
17,coffee_bar,0.015061


In [62]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [63]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 0.0
Mean squared error for test data: 561018.1533460533


In [64]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [14225074.86054105 15694834.63063942 13125976.99949116]


In [65]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

13125976.999491159 15694834.630639417 14348628.830223873


# random forest regressor

In [66]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [67]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False).head(30)

,feature,importance
3,media_type,0.309723
40,promotion_period,0.207688
2,promotion_name,0.092527
34,grocery_sqft,0.056146
6,state_province,0.048087
35,frozen_sqft,0.044153
33,store_sqft,0.042030
36,meat_sqft,0.040801
5,store_type,0.028211
4,sales_region,0.019006


In [68]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [69]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 44409.66602783215
Mean squared error for test data: 335269.3697632605


In [70]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10390295.87984292 12126272.15994684 11791752.44399546]


In [71]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10390295.879842917 12126272.15994684 11436106.827928407


# Ada Boost regressor

In [72]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [73]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False).head(30)

,feature,importance
3,media_type,0.359308
40,promotion_period,0.274877
4,sales_region,0.096897
6,state_province,0.086330
5,store_type,0.064592
34,grocery_sqft,0.038300
33,store_sqft,0.033604
2,promotion_name,0.016489
35,frozen_sqft,0.015140
18,video_store,0.008783


In [74]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [75]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 7866004.848954066
Mean squared error for test data: 7893522.383171979


In [76]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 8785694.6893474  10352301.85061023  9147555.94003414]


In [98]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8909614.316433514 9601003.829353709 9153231.66058074


# Gradient boost regressor

In [77]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [78]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False).head(30)

,feature,importance
3,media_type,2.904456e-01
40,promotion_period,1.527473e-01
2,promotion_name,1.304930e-01
34,grocery_sqft,1.086911e-01
33,store_sqft,6.322689e-02
6,state_province,5.239817e-02
5,store_type,4.268220e-02
35,frozen_sqft,4.216660e-02
4,sales_region,3.187172e-02
17,coffee_bar,2.687775e-02


In [79]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [80]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5161114.369448772
Mean squared error for test data: 5229107.528719718


In [81]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10014034.24520198 11046663.35474382 11080181.04795325]


In [82]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10014034.245201983 11080181.047953254 10713626.215966351


# XGBoost Regressor

In [83]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [84]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False).head(30)

,feature,importance
2,promotion_name,0.102267
35,frozen_sqft,0.093978
33,store_sqft,0.091026
40,promotion_period,0.087265
6,state_province,0.086315
34,grocery_sqft,0.084142
18,video_store,0.078803
17,coffee_bar,0.074884
3,media_type,0.072555
5,store_type,0.056028


In [85]:
df_imp_xgbr.sort_values(by="importance",ascending=False).head(30)["feature"].values

array(['promotion_name', 'frozen_sqft', 'store_sqft', 'promotion_period',
       'state_province', 'grocery_sqft', 'video_store', 'coffee_bar',
       'media_type', 'store_type', 'sales_region', 'salad_bar', 'country',
       'florist', 'prepared_food', 'unit_sales', 'member_card',
       'num_cars_owned', 'education', 'occupation', 'gender',
       'marital_status', 'yearly_income', 'houseowner',
       'num_children_at_home', 'total_children', 'store_sales',
       'recyclable_package', 'cases_per_pallet', 'units_per_case'],
      dtype=object)

In [86]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [87]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 428792.3654407066
Mean squared error for test data: 554400.562276955


In [88]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9939227.87247383 12553596.39849884 12393914.45307676]


In [89]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9939227.872473828 12553596.398498835 11628912.908016473


# StackingRegressor

In [90]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [91]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [92]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 351840.2345759717
Mean squared error for test data: 673754.8133700798


In [93]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [16358673.02052239 16817513.32615332 16685076.448077  ]


In [94]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

16358673.020522388 16817513.326153323 16620420.931584239


In [95]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [96]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [97]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 40711.2013441268
Mean squared error for test data: 425697.52649419097


In [98]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9825852.84437458 12345565.8774504  12836783.41904452]


In [99]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9825852.844374578 12836783.419044515 11669400.713623164


In [ ]:
df1.columns

In [100]:
li=['store_sales', 'store_cost', 'unit_sales','brand_name', 'product_name','SRP', 'gross_weight',
       'net_weight', 'recyclable_package', 'low_fat', 'units_per_case',
       'cases_per_pallet', 'shelf_width', 'shelf_height', 'shelf_depth',
       'subcategory', 'category', 'department', 'family','promotion_name', 'media_type', 'cost','sales_city',
       'sales_state_province', 'sales_district', 'sales_region',
       'sales_country','store_type', 'store_name','store_city', 'store_state','store_country', 'store_manager','store_sqft',
       'grocery_sqft', 'frozen_sqft', 'meat_sqft', 'coffee_bar', 'video_store',
       'salad_bar', 'prepared_food', 'florist','city',
       'state_province','country','marital_status', 'yearly_income',
       'gender', 'total_children', 'num_children_at_home', 'education','member_card', 'occupation', 'houseowner',
       'num_cars_owned','promotion_period']

In [101]:
len(li)

56

In [ ]:
['recyclable_package', 'low_fat', 'department', 'family', 'promotion_name', 'media_type', 'sales_district', 
'sales_region', 'store_type', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist', 'state_province',
'country', 'marital_status', 'yearly_income', 'gender', 'education','member_card', 'occupation', 'houseowner','store_sales',
'store_cost', 'unit_sales', 'SRP', 'gross_weight', 'net_weight', 'units_per_case', 'cases_per_pallet', 'shelf_width',
'shelf_height', 'shelf_depth', 'cost', 'store_sqft', 'grocery_sqft', 'frozen_sqft', 'meat_sqft', 'total_children',
'num_children_at_home', 'num_cars_owned', 'promotion_period']

# selecting 31 columns based on domain

In [84]:
df1['sales_district']

0        Bremerton
1        Bremerton
2        Bremerton
3        Bremerton
4        Bremerton
           ...    
70746     Victoria
70747     Victoria
70748     Victoria
70749     Victoria
70750     Victoria
Name: sales_district, Length: 70751, dtype: object

In [86]:
df1['sales_region']

0         North West
1         North West
2         North West
3         North West
4         North West
            ...     
70746    Canada West
70747    Canada West
70748    Canada West
70749    Canada West
70750    Canada West
Name: sales_region, Length: 70751, dtype: object

In [87]:
df1['state_province']

0        WA
1        WA
2        WA
3        WA
4        WA
         ..
70746    BC
70747    BC
70748    BC
70749    BC
70750    BC
Name: state_province, Length: 70751, dtype: object

* selecting a few out of the location columns, omitting few store space related, shelf related,consumer related and cases related columns

In [102]:
l=['recyclable_package', 'low_fat', 'department', 'family', 'promotion_name', 'media_type', 'sales_region', 
'store_type', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist',
'country', 'marital_status', 'yearly_income', 'gender', 'education','member_card', 'occupation', 'store_sales',
'store_cost', 'unit_sales', 'SRP', 'gross_weight', 'net_weight', 'cost', 'store_sqft', 'grocery_sqft', 'total_children',
'promotion_period']

In [103]:
len(l)

31

In [437]:
df_fresh=df1[['recyclable_package', 'low_fat', 'department', 'family', 'promotion_name', 'media_type', 'sales_region', 
'store_type', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist',
'country', 'marital_status', 'yearly_income', 'gender', 'education','member_card', 'occupation', 'store_sales',
'store_cost', 'unit_sales', 'SRP', 'gross_weight', 'net_weight', 'cost', 'store_sqft', 'grocery_sqft', 'total_children',
'promotion_period']]

In [438]:
df_fresh["department"] = df_fresh["department"].apply(func_dept)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\11639750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fresh["department"] = df_fresh["department"].apply(func_dept)


In [439]:
df_fresh['promotion_name']=df_fresh['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\1922926961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fresh['promotion_name']=df_fresh['promotion_name'].apply(func_prom_name)


In [440]:
le=LabelEncoder()
df_fresh_num=df_fresh.select_dtypes(include=np.number)
df_fresh_cat=df_fresh.select_dtypes(exclude=np.number)

In [441]:
df_fresh_num

,recyclable_package,low_fat,coffee_bar,video_store,salad_bar,prepared_food,florist,store_sales,store_cost,unit_sales,SRP,gross_weight,net_weight,cost,store_sqft,grocery_sqft,total_children,promotion_period
0,1,0,0,0,1,1,0,7.12,2.2784,4,1.78,13.40,11.30,12950,39696.0,24390.0,2,4
1,0,1,0,0,1,1,0,5.76,1.7856,4,1.44,8.25,5.25,12950,39696.0,24390.0,2,4
2,1,0,0,0,1,1,0,5.55,2.0535,3,1.85,7.12,4.11,12950,39696.0,24390.0,2,4
3,0,1,0,0,1,1,0,7.48,2.2440,2,3.74,13.10,12.10,12950,39696.0,24390.0,2,4
4,1,0,0,0,1,1,0,2.28,1.0488,4,0.57,16.90,14.80,12950,39696.0,24390.0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,0,1,1,0,0,0,1,7.68,3.5328,3,2.56,10.50,8.50,13006,34452.0,27463.0,4,3
70747,1,0,1,0,0,0,1,2.85,0.8835,5,0.57,9.30,7.30,13006,34452.0,27463.0,4,3
70748,0,0,1,0,0,0,1,4.29,1.4586,3,1.43,10.20,7.19,13006,34452.0,27463.0,1,3
70749,1,1,1,0,0,0,1,5.18,2.0720,2,2.59,9.77,7.77,13006,34452.0,27463.0,1,3


In [442]:
df_fresh_cat

,department,family,promotion_name,media_type,sales_region,store_type,country,marital_status,yearly_income,gender,education,member_card,occupation
0,Household,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual
1,Dairy,Food,Sale_discount,Cash Register Handout,North West,Supermarket,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual
2,Health and Hygiene,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual
3,Produce,Food,Sale_discount,Cash Register Handout,North West,Supermarket,USA,S,$50K - $70K,M,High School Degree,Normal,Skilled Manual
4,Ready_to_eat,Food,Sale_discount,Cash Register Handout,North West,Supermarket,USA,S,$50K - $70K,M,High School Degree,Normal,Skilled Manual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,Canada,S,$110K - $130K,M,High School Degree,Silver,Management
70747,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,Canada,S,$110K - $130K,M,High School Degree,Silver,Management
70748,Household,Non-Consumable,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,Canada,S,$90K - $110K,M,Bachelors Degree,Golden,Professional
70749,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,Canada,S,$90K - $110K,M,Bachelors Degree,Golden,Professional


In [443]:
for col in df_fresh_cat.columns:
    df_fresh_cat[col] = le.fit_transform(df_fresh_cat[col])

In [444]:
df_fresh_cat

,department,family,promotion_name,media_type,sales_region,store_type,country,marital_status,yearly_income,gender,education,member_card,occupation
0,7,2,2,1,5,4,2,0,4,0,2,0,4
1,4,1,2,1,5,4,2,0,4,0,2,0,4
2,6,2,2,1,5,4,2,0,4,0,2,0,4
3,10,1,2,1,5,4,2,1,5,1,2,2,4
4,11,1,2,1,5,4,2,1,5,1,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,10,1,2,5,0,2,0,1,1,1,2,3,1
70747,10,1,2,5,0,2,0,1,1,1,2,3,1
70748,7,2,2,5,0,2,0,1,7,1,0,1,3
70749,10,1,2,5,0,2,0,1,7,1,0,1,3


In [445]:
df_fresh=pd.concat([df_fresh_cat,df_fresh_num],axis=1)

In [446]:
#Setting target variable
X = df_fresh.drop(columns=['cost'])
y = df_fresh['cost']

In [447]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression

In [448]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [449]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     69.47
Date:                Mon, 26 Feb 2024   Prob (F-statistic):               0.00
Time:                        01:04:40   Log-Likelihood:            -5.3189e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56569   BIC:                         1.064e+06
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               1.373e+04    209.549     65.522      0.000    1.33e+04    1.41e+04
department            -0.8300      3.179     -0.261      0.794      -7.060       5.400
family               -22.4718     24.403     -0.921      0.357     -70.301      25.357
promotion_name       -83.7489     12.909     -6.488      0.000    -109.050     -58.447
media_type           -25.4938      3.591     -7.100      0.000     -32.531     -18.456
sales_region        -154.7877     23.521     -6.581      0.000    -200.889    -108.686
store_type          -202.7306     18.249    -11.109      0.000    -238.498    -166.963
country               -0.8394     64.985     -0.013      0.990    -128.211     126.532
marital_status        36.5437     24.647      1.483      0.138     -11.765      84.852
yearly_income          1.4035      7.523      0.187      0.852     -13.341      16.148
gender               -28.5743     24.627     -1.160      0.246     -76.844      19.695
education             -9.0930      9.836     -0.924      0.355     -28.372      10.186
member_card            1.2368     13.432      0.092      0.927     -25.089      27.563
occupation            -5.0922     11.693     -0.436      0.663     -28.010      17.825
recyclable_package     2.8963     24.770      0.117      0.907     -45.654      51.446
low_fat                2.9147     26.409      0.110      0.912     -48.847      54.677
coffee_bar          -356.8054     48.220     -7.400      0.000    -451.316    -262.295
video_store         -773.1731     42.524    -18.182      0.000    -856.521    -689.825
salad_bar            175.6222     85.919      2.044      0.041       7.220     344.025
prepared_food       -224.5723     81.760     -2.747      0.006    -384.823     -64.322
florist             -437.2306     54.791     -7.980      0.000    -544.622    -329.840
store_sales           -5.4203     19.383     -0.280      0.780     -43.411      32.570
store_cost            33.4735     28.202      1.187      0.235     -21.802      88.749
unit_sales           -29.3390     36.546     -0.803      0.422    -100.969      42.291
SRP                  -17.3799     50.439     -0.345      0.730    -116.241      81.481
gross_weight           9.9426     17.882      0.556      0.578     -25.107      44.992
net_weight           -11.0399     17.663     -0.625      0.532     -45.659      23.580
store_sqft             0.0036      0.007      0.545      0.585      -0.009       0.017
grocery_sqft          -0.0732      0.010     -7.465      0.000      -0.092      -0.054
total_children       -25.2886      8.364     -3.024      0.002     -41.682      -8.895
promotion_period     -66.9777      3.299    -20.301      0.000     -73.444     -60.511
==============================================================================
Omnibus:                    35979.851   Durbin

In [450]:
significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const               0.000000e+00
promotion_name      8.787978e-11
media_type          1.260394e-12
sales_region        4.720820e-11
store_type          1.209813e-28
coffee_bar          1.384975e-13
video_store         1.163384e-73
salad_bar           4.095457e-02
prepared_food       6.021359e-03
florist             1.491284e-15
grocery_sqft        8.466881e-14
total_children      2.499456e-03
promotion_period    2.685694e-91
dtype: float64


In [451]:
sig_pvalues_features=significant_pvalues.index

In [452]:
sig_pvalues_features.drop("const")

Index(['promotion_name', 'media_type', 'sales_region', 'store_type',
       'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist',
       'grocery_sqft', 'total_children', 'promotion_period'],
      dtype='object')

In [453]:
len(significant_pvalues)

13

In [454]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [455]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8509177.272296203
Mean squared error for test data: 8517338.395818675


In [456]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [9263275.41126378 9702152.7445999  8926274.40897566]


In [457]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8926274.408975665 9702152.744599901 9297234.188279781


In [458]:
lr=LinearRegression()
model_sfs=sfs(estimator=lr,k_features="best",forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'marital_status',
 'gender',
 'education',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'prepared_food',
 'florist',
 'store_cost',
 'unit_sales',
 'grocery_sqft',
 'total_children',
 'promotion_period')

In [459]:
len(model_sfs.k_feature_names_)

17

In [460]:
lr=LinearRegression()
model_sfs=sfs(estimator=lr,k_features=15,forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'marital_status',
 'gender',
 'education',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'prepared_food',
 'florist',
 'grocery_sqft',
 'total_children',
 'promotion_period')

In [461]:
len(model_sfs.k_feature_names_)

15

# Decision Tree Regressor

In [462]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [463]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False).head(15)

,feature,importance
3,media_type,0.318134
29,promotion_period,0.168378
2,promotion_name,0.124962
26,store_sqft,0.073749
27,grocery_sqft,0.068639
5,store_type,0.042976
4,sales_region,0.034666
19,florist,0.029890
15,coffee_bar,0.021230
16,video_store,0.020889


In [464]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [465]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 0.0
Mean squared error for test data: 630202.7245424351


In [466]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [13874702.45509668 18287183.34252035 11869156.45036679]


In [467]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

11869156.45036679 18287183.342520352 14677014.082661271


# Random Forest Regressor

In [468]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [469]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False).head(15)

,feature,importance
3,media_type,0.305393
29,promotion_period,0.210716
27,grocery_sqft,0.094876
2,promotion_name,0.093977
26,store_sqft,0.083158
5,store_type,0.038598
4,sales_region,0.032592
16,video_store,0.022607
15,coffee_bar,0.016095
19,florist,0.014143


In [470]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [471]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 45585.105887399295
Mean squared error for test data: 367768.9320454808


In [472]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10459392.65976657 13111795.57536581 11606229.57966368]


In [473]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10459392.65976657 13111795.575365812 11725805.938265353


# Ada Boost Regressor

In [474]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [475]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
3,media_type,0.367437
29,promotion_period,0.232559
27,grocery_sqft,0.181981
4,sales_region,0.148856
26,store_sqft,0.038347
5,store_type,0.029713
2,promotion_name,0.001106
28,total_children,0.000000
17,salad_bar,0.000000
25,net_weight,0.000000


In [476]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [477]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 7957667.719402413
Mean squared error for test data: 7996322.291602776


In [478]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [8704698.6527967  9987186.86845405 9011188.46382627]


In [479]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8704698.652796702 9987186.86845405 9234357.995025676


# Gradient Boost Regressor

In [480]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [481]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
3,media_type,0.300169
29,promotion_period,0.168129
27,grocery_sqft,0.157234
2,promotion_name,0.116697
26,store_sqft,0.107223
5,store_type,0.056047
4,sales_region,0.032369
15,coffee_bar,0.026017
16,video_store,0.017345
18,prepared_food,0.005506


In [482]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [483]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5187758.729879312
Mean squared error for test data: 5236394.9905657405


In [484]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9548357.10332978 11869568.1004991  10592465.37951063]


In [485]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9548357.103329776 11869568.100499105 10670130.194446502


# XGBoostRegressor

In [486]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [487]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
16,video_store,0.116878
29,promotion_period,0.109005
2,promotion_name,0.108307
26,store_sqft,0.083639
27,grocery_sqft,0.082974
17,salad_bar,0.080186
3,media_type,0.077942
15,coffee_bar,0.071782
5,store_type,0.064980
4,sales_region,0.057707


In [488]:
df_imp_xgbr.sort_values(by="importance",ascending=False).head(15)["feature"].values

array(['video_store', 'promotion_period', 'promotion_name', 'store_sqft',
       'grocery_sqft', 'salad_bar', 'media_type', 'coffee_bar',
       'store_type', 'sales_region', 'florist', 'prepared_food',
       'country', 'marital_status', 'member_card'], dtype=object)

In [489]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [490]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 446130.4611248761
Mean squared error for test data: 557111.3912922835


In [491]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9345270.75081274 12664766.13839956 12297805.24723431]


In [492]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9345270.75081274 12664766.138399558 11435947.378815534


# Stacking Regressor

In [493]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [494]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [495]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 438280.9319964664
Mean squared error for test data: 677487.629001484


In [496]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [16454706.63992537 14910529.91345828 15555322.64173345]


In [497]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

14910529.913458277 16454706.639925374 15640186.398372367


In [498]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [499]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [500]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 30408.65530587768
Mean squared error for test data: 411485.59181351925


In [501]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [14223620.60483915 12372570.96817748 11394016.31834943]


In [502]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

11394016.318349428 14223620.604839152 12663402.630455352


# selecting 31 columns using SFS

In [172]:
df_fresh_sfs=df1[['family',
 'promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'state_province',
 'marital_status',
 'yearly_income',
 'gender',
 'education',
 'occupation',
 'houseowner',
 'low_fat',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'prepared_food',
 'florist',
 'store_sales',
 'store_cost',
 'unit_sales',
 'gross_weight',
 'net_weight',
 'store_sqft',
 'grocery_sqft',
 'frozen_sqft',
 'meat_sqft',
 'total_children',
 'num_cars_owned',
 'promotion_period',"cost"]]

In [174]:
df_fresh_sfs['promotion_name']=df_fresh_sfs['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\2368060727.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fresh_sfs['promotion_name']=df_fresh_sfs['promotion_name'].apply(func_prom_name)


In [175]:
df_fresh_sfs

,family,promotion_name,media_type,sales_region,store_type,state_province,marital_status,yearly_income,gender,education,...,gross_weight,net_weight,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,total_children,num_cars_owned,promotion_period,cost
0,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,WA,M,$30K - $50K,F,High School Degree,...,13.40,11.30,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
1,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,M,$30K - $50K,F,High School Degree,...,8.25,5.25,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
2,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,WA,M,$30K - $50K,F,High School Degree,...,7.12,4.11,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
3,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,S,$50K - $70K,M,High School Degree,...,13.10,12.10,39696.0,24390.0,9184.0,6122.0,2,2,4,12950
4,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,S,$50K - $70K,M,High School Degree,...,16.90,14.80,39696.0,24390.0,9184.0,6122.0,2,2,4,12950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,S,$110K - $130K,M,High School Degree,...,10.50,8.50,34452.0,27463.0,4193.0,2795.0,4,3,3,13006
70747,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,S,$110K - $130K,M,High School Degree,...,9.30,7.30,34452.0,27463.0,4193.0,2795.0,4,3,3,13006
70748,Non-Consumable,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,S,$90K - $110K,M,Bachelors Degree,...,10.20,7.19,34452.0,27463.0,4193.0,2795.0,1,2,3,13006
70749,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,S,$90K - $110K,M,Bachelors Degree,...,9.77,7.77,34452.0,27463.0,4193.0,2795.0,1,2,3,13006


In [176]:
le=LabelEncoder()
df_fresh_sfs_num=df_fresh_sfs.select_dtypes(include=np.number)
df_fresh_sfs_cat=df_fresh_sfs.select_dtypes(exclude=np.number)

In [177]:
df_fresh_sfs_num

,low_fat,coffee_bar,video_store,salad_bar,prepared_food,florist,store_sales,store_cost,unit_sales,gross_weight,net_weight,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,total_children,num_cars_owned,promotion_period,cost
0,0,0,0,1,1,0,7.12,2.2784,4,13.40,11.30,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
1,1,0,0,1,1,0,5.76,1.7856,4,8.25,5.25,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
2,0,0,0,1,1,0,5.55,2.0535,3,7.12,4.11,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
3,1,0,0,1,1,0,7.48,2.2440,2,13.10,12.10,39696.0,24390.0,9184.0,6122.0,2,2,4,12950
4,0,0,0,1,1,0,2.28,1.0488,4,16.90,14.80,39696.0,24390.0,9184.0,6122.0,2,2,4,12950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,1,1,0,0,0,1,7.68,3.5328,3,10.50,8.50,34452.0,27463.0,4193.0,2795.0,4,3,3,13006
70747,0,1,0,0,0,1,2.85,0.8835,5,9.30,7.30,34452.0,27463.0,4193.0,2795.0,4,3,3,13006
70748,0,1,0,0,0,1,4.29,1.4586,3,10.20,7.19,34452.0,27463.0,4193.0,2795.0,1,2,3,13006
70749,1,1,0,0,0,1,5.18,2.0720,2,9.77,7.77,34452.0,27463.0,4193.0,2795.0,1,2,3,13006


In [178]:
df_fresh_sfs_cat

,family,promotion_name,media_type,sales_region,store_type,state_province,marital_status,yearly_income,gender,education,occupation,houseowner
0,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,WA,M,$30K - $50K,F,High School Degree,Skilled Manual,Y
1,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,M,$30K - $50K,F,High School Degree,Skilled Manual,Y
2,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,WA,M,$30K - $50K,F,High School Degree,Skilled Manual,Y
3,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,S,$50K - $70K,M,High School Degree,Skilled Manual,Y
4,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,S,$50K - $70K,M,High School Degree,Skilled Manual,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
70746,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,S,$110K - $130K,M,High School Degree,Management,Y
70747,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,S,$110K - $130K,M,High School Degree,Management,Y
70748,Non-Consumable,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,S,$90K - $110K,M,Bachelors Degree,Professional,Y
70749,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,S,$90K - $110K,M,Bachelors Degree,Professional,Y


In [179]:
for col in df_fresh_sfs_cat.columns:
    df_fresh_sfs_cat[col] = le.fit_transform(df_fresh_sfs_cat[col])

In [180]:
  df_fresh_sfs_cat

,family,promotion_name,media_type,sales_region,store_type,state_province,marital_status,yearly_income,gender,education,occupation,houseowner
0,2,2,1,5,4,9,0,4,0,2,4,1
1,1,2,1,5,4,9,0,4,0,2,4,1
2,2,2,1,5,4,9,0,4,0,2,4,1
3,1,2,1,5,4,9,1,5,1,2,4,1
4,1,2,1,5,4,9,1,5,1,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
70746,1,2,5,0,2,0,1,1,1,2,1,1
70747,1,2,5,0,2,0,1,1,1,2,1,1
70748,2,2,5,0,2,0,1,7,1,0,3,1
70749,1,2,5,0,2,0,1,7,1,0,3,1


In [181]:
df_fresh_sfs=pd.concat([df_fresh_sfs_cat,df_fresh_sfs_num],axis=1)

In [182]:
#Setting target variable
X = df_fresh_sfs.drop(columns=['cost'])
y = df_fresh_sfs['cost']

In [183]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression

In [184]:
df_fresh_sfs

,family,promotion_name,media_type,sales_region,store_type,state_province,marital_status,yearly_income,gender,education,...,gross_weight,net_weight,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,total_children,num_cars_owned,promotion_period,cost
0,2,2,1,5,4,9,0,4,0,2,...,13.40,11.30,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
1,1,2,1,5,4,9,0,4,0,2,...,8.25,5.25,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
2,2,2,1,5,4,9,0,4,0,2,...,7.12,4.11,39696.0,24390.0,9184.0,6122.0,2,4,4,12950
3,1,2,1,5,4,9,1,5,1,2,...,13.10,12.10,39696.0,24390.0,9184.0,6122.0,2,2,4,12950
4,1,2,1,5,4,9,1,5,1,2,...,16.90,14.80,39696.0,24390.0,9184.0,6122.0,2,2,4,12950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,1,2,5,0,2,0,1,1,1,2,...,10.50,8.50,34452.0,27463.0,4193.0,2795.0,4,3,3,13006
70747,1,2,5,0,2,0,1,1,1,2,...,9.30,7.30,34452.0,27463.0,4193.0,2795.0,4,3,3,13006
70748,2,2,5,0,2,0,1,7,1,0,...,10.20,7.19,34452.0,27463.0,4193.0,2795.0,1,2,3,13006
70749,1,2,5,0,2,0,1,7,1,0,...,9.77,7.77,34452.0,27463.0,4193.0,2795.0,1,2,3,13006


In [185]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [186]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     80.92
Date:                Sun, 25 Feb 2024   Prob (F-statistic):               0.00
Time:                        22:41:26   Log-Likelihood:            -5.3172e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56569   BIC:                         1.064e+06
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.606e+04    258.006     62.245      0.000    1.56e+04    1.66e+04
family             -24.5911     23.751     -1.035      0.300     -71.142      21.960
promotion_name     -82.9018     12.937     -6.408      0.000    -108.259     -57.544
media_type         -21.1168      3.605     -5.857      0.000     -28.183     -14.050
sales_region      -209.1230     10.740    -19.471      0.000    -230.174    -188.072
store_type        -507.1061     27.827    -18.223      0.000    -561.648    -452.564
state_province     -51.0155      4.716    -10.818      0.000     -60.258     -41.773
marital_status      44.9931     25.591      1.758      0.079      -5.164      95.151
yearly_income       -7.2208      7.465     -0.967      0.333     -21.852       7.410
gender             -33.6370     24.590     -1.368      0.171     -81.833      14.559
education          -10.3264      9.822     -1.051      0.293     -29.577       8.924
occupation           2.2618     11.671      0.194      0.846     -20.614      25.138
houseowner          39.2134     26.125      1.501      0.133     -11.992      90.419
low_fat              0.3413     26.023      0.013      0.990     -50.664      51.347
coffee_bar        -597.8831     56.904    -10.507      0.000    -709.415    -486.351
video_store      -1268.5751     52.803    -24.025      0.000   -1372.070   -1165.081
salad_bar         1482.1200    150.411      9.854      0.000    1187.314    1776.926
prepared_food    -1542.7270    147.315    -10.472      0.000   -1831.465   -1253.989
florist          -1181.0018     68.142    -17.331      0.000   -1314.560   -1047.443
store_sales        -11.0559     11.975     -0.923      0.356     -34.527      12.415
store_cost          35.2645     28.119      1.254      0.210     -19.850      90.379
unit_sales          -9.5747     17.358     -0.552      0.581     -43.597      24.448
gross_weight         7.7465     17.827      0.435      0.664     -27.194      42.687
net_weight          -8.8289     17.604     -0.502      0.616     -43.333      25.675
store_sqft          -0.5473      0.102     -5.387      0.000      -0.746      -0.348
grocery_sqft         0.4577      0.099      4.600      0.000       0.263       0.653
frozen_sqft        593.3003     37.033     16.021      0.000     520.715     665.886
meat_sqft         -888.6544     55.502    -16.011      0.000    -997.439    -779.870
total_children     -30.5918      8.361     -3.659      0.000     -46.980     -14.204
num_cars_owned      48.1686     12.790      3.766      0.000      23.101      73.236
promotion_period   -59.1538      3.313    -17.855      0.000     -65.647     -52.660
==============================================================================
Omnibus:                    29702.517   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.0

In [187]:
significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const                0.000000e+00
promotion_name       1.486805e-10
media_type           4.731478e-09
sales_region         3.687185e-84
store_type           5.497871e-74
state_province       3.001637e-27
coffee_bar           8.480101e-26
video_store         6.676964e-127
salad_bar            6.885612e-23
prepared_food        1.222128e-25
florist              4.060814e-67
store_sqft           7.211416e-08
grocery_sqft         4.232279e-06
frozen_sqft          1.225763e-57
meat_sqft            1.430297e-57
total_children       2.536169e-04
num_cars_owned       1.658964e-04
promotion_period     4.173818e-71
dtype: float64


In [188]:
sig_pvalues_features=significant_pvalues.index

In [189]:
sig_pvalues_features.drop("const")

Index(['promotion_name', 'media_type', 'sales_region', 'store_type',
       'state_province', 'coffee_bar', 'video_store', 'salad_bar',
       'prepared_food', 'florist', 'store_sqft', 'grocery_sqft', 'frozen_sqft',
       'meat_sqft', 'total_children', 'num_cars_owned', 'promotion_period'],
      dtype='object')

In [190]:
len(significant_pvalues)

18

In [191]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [192]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8459617.266401043
Mean squared error for test data: 8465454.810869714


In [193]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10006384.6262394   9647152.97636066  8911748.74386643]


In [194]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8911748.743866432 10006384.626239398 9521762.115488831


In [195]:
lr=LinearRegression()
model_sfs=sfs(estimator=lr,k_features="best",forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('family',
 'promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'state_province',
 'marital_status',
 'gender',
 'houseowner',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'prepared_food',
 'florist',
 'store_cost',
 'unit_sales',
 'store_sqft',
 'grocery_sqft',
 'frozen_sqft',
 'meat_sqft',
 'total_children',
 'num_cars_owned',
 'promotion_period')

In [196]:
len(model_sfs.k_feature_names_)

23

In [197]:
lr=LinearRegression()
model_sfs=sfs(estimator=lr,k_features=15,forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'state_province',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'prepared_food',
 'florist',
 'store_sqft',
 'grocery_sqft',
 'frozen_sqft',
 'meat_sqft',
 'promotion_period')

In [198]:
len(model_sfs.k_feature_names_)

15

# Decision Tree Regressor

In [199]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [200]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.297890
29,promotion_period,0.165132
1,promotion_name,0.141867
24,grocery_sqft,0.072585
5,state_province,0.043966
26,meat_sqft,0.042666
23,store_sqft,0.031939
25,frozen_sqft,0.029364
4,store_type,0.029094
16,prepared_food,0.024204


In [201]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [202]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 0.0
Mean squared error for test data: 409255.8821284715


In [203]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [13198620.27480495 15863110.01903833 13430084.9046771 ]


In [204]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

13198620.274804952 15863110.01903833 14163938.399506794


# Random Forest Regressor

In [205]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [206]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.309748
29,promotion_period,0.208252
1,promotion_name,0.092636
24,grocery_sqft,0.058698
5,state_province,0.049016
25,frozen_sqft,0.042629
23,store_sqft,0.041378
26,meat_sqft,0.041354
4,store_type,0.029506
3,sales_region,0.023095


In [207]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [208]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 36759.44216839752
Mean squared error for test data: 287223.0837253268


In [209]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10549788.44073653 12226938.41079788 11802211.38488543]


In [210]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10549788.440736532 12226938.410797877 11526312.74547328


# Ada Boost Regressor

In [211]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [212]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.359308
29,promotion_period,0.234277
3,sales_region,0.096897
5,state_province,0.086330
4,store_type,0.064592
1,promotion_name,0.057088
24,grocery_sqft,0.038300
23,store_sqft,0.033604
26,meat_sqft,0.013559
14,video_store,0.008783


In [213]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [214]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 7866004.848954066
Mean squared error for test data: 7893522.383171979


In [215]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 8785694.6893474  10306386.89581476  9147555.94003414]


In [216]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8785694.689347396 10306386.895814763 9413212.508398768


# Gradient Boost Regressor

In [217]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [218]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.287702
29,promotion_period,0.145285
1,promotion_name,0.139908
24,grocery_sqft,0.105809
23,store_sqft,0.073771
5,state_province,0.055829
4,store_type,0.048151
3,sales_region,0.030515
25,frozen_sqft,0.028752
13,coffee_bar,0.023853


In [219]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [220]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5214838.770403404
Mean squared error for test data: 5284170.785562899


In [221]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10094434.27572904 11323222.44538574 11070780.94342205]


In [222]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10094434.275729043 11323222.445385745 10829479.22151228


# XGBoost Regressor

In [223]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [224]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,promotion_name,0.099056
25,frozen_sqft,0.096026
29,promotion_period,0.096018
13,coffee_bar,0.095641
24,grocery_sqft,0.092249
5,state_province,0.089163
23,store_sqft,0.086966
2,media_type,0.081908
14,video_store,0.065522
4,store_type,0.054151


In [225]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [226]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 429826.0116460705
Mean squared error for test data: 543929.3498368684


In [227]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10122363.40308362 11777801.27534577 13225524.1744175 ]


In [228]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10122363.403083615 13225524.1744175 11708562.950948963


# Stacking Regressor

In [229]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [230]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [231]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 280023.1122968198
Mean squared error for test data: 590659.7976821426


In [232]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [15788478.48058005 16840877.34697253 15886688.31463342]


In [233]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

15788478.480580054 16840877.346972525 16172014.714062


In [234]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [235]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [236]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 24485.275923104276
Mean squared error for test data: 323916.36213980144


In [237]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [11455468.64674451 13996337.40684376 12893479.81122011]


In [238]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

11455468.64674451 13996337.40684376 12781761.954936126


# selecting 31 columns using rel imp of xgb model for 43 columns

In [239]:
df_fresh_xgb=df1[['promotion_name', 'frozen_sqft', 'store_sqft', 'promotion_period',
       'state_province', 'grocery_sqft', 'video_store', 'coffee_bar',
       'media_type', 'store_type', 'sales_region', 'salad_bar', 'country',
       'florist', 'prepared_food', 'unit_sales', 'member_card',
       'num_cars_owned', 'education', 'occupation', 'gender',
       'marital_status', 'yearly_income', 'houseowner',
       'num_children_at_home', 'total_children', 'store_sales',
       'recyclable_package', 'cases_per_pallet', 'units_per_case',"cost"]]

In [240]:
df_fresh_xgb

,promotion_name,frozen_sqft,store_sqft,promotion_period,state_province,grocery_sqft,video_store,coffee_bar,media_type,store_type,...,marital_status,yearly_income,houseowner,num_children_at_home,total_children,store_sales,recyclable_package,cases_per_pallet,units_per_case,cost
0,Super Duper Savers,9184.0,39696.0,4,WA,24390.0,0,0,Cash Register Handout,Supermarket,...,M,$30K - $50K,Y,2,2,7.12,1,11,25,12950
1,Super Duper Savers,9184.0,39696.0,4,WA,24390.0,0,0,Cash Register Handout,Supermarket,...,M,$30K - $50K,Y,2,2,5.76,0,6,22,12950
2,Super Duper Savers,9184.0,39696.0,4,WA,24390.0,0,0,Cash Register Handout,Supermarket,...,M,$30K - $50K,Y,2,2,5.55,1,8,34,12950
3,Super Duper Savers,9184.0,39696.0,4,WA,24390.0,0,0,Cash Register Handout,Supermarket,...,S,$50K - $70K,Y,0,2,7.48,0,13,21,12950
4,Super Duper Savers,9184.0,39696.0,4,WA,24390.0,0,0,Cash Register Handout,Supermarket,...,S,$50K - $70K,Y,0,2,2.28,1,10,6,12950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,Savings Galore,4193.0,34452.0,3,BC,27463.0,0,1,In-Store Coupon,Mid-Size Grocery,...,S,$110K - $130K,Y,0,4,7.68,0,13,12,13006
70747,Savings Galore,4193.0,34452.0,3,BC,27463.0,0,1,In-Store Coupon,Mid-Size Grocery,...,S,$110K - $130K,Y,0,4,2.85,1,12,24,13006
70748,Savings Galore,4193.0,34452.0,3,BC,27463.0,0,1,In-Store Coupon,Mid-Size Grocery,...,S,$90K - $110K,Y,0,1,4.29,0,7,5,13006
70749,Savings Galore,4193.0,34452.0,3,BC,27463.0,0,1,In-Store Coupon,Mid-Size Grocery,...,S,$90K - $110K,Y,0,1,5.18,1,10,20,13006


In [241]:
df_fresh_xgb['promotion_name']=df_fresh_xgb['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\4125500647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fresh_xgb['promotion_name']=df_fresh_xgb['promotion_name'].apply(func_prom_name)


In [242]:
le=LabelEncoder()
df_fresh_xgb_num=df_fresh_xgb.select_dtypes(include=np.number)
df_fresh_xgb_cat=df_fresh_xgb.select_dtypes(exclude=np.number)

In [243]:
df_fresh_xgb_num

,frozen_sqft,store_sqft,promotion_period,grocery_sqft,video_store,coffee_bar,salad_bar,florist,prepared_food,unit_sales,num_cars_owned,num_children_at_home,total_children,store_sales,recyclable_package,cases_per_pallet,units_per_case,cost
0,9184.0,39696.0,4,24390.0,0,0,1,0,1,4,4,2,2,7.12,1,11,25,12950
1,9184.0,39696.0,4,24390.0,0,0,1,0,1,4,4,2,2,5.76,0,6,22,12950
2,9184.0,39696.0,4,24390.0,0,0,1,0,1,3,4,2,2,5.55,1,8,34,12950
3,9184.0,39696.0,4,24390.0,0,0,1,0,1,2,2,0,2,7.48,0,13,21,12950
4,9184.0,39696.0,4,24390.0,0,0,1,0,1,4,2,0,2,2.28,1,10,6,12950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,4193.0,34452.0,3,27463.0,0,1,0,1,0,3,3,0,4,7.68,0,13,12,13006
70747,4193.0,34452.0,3,27463.0,0,1,0,1,0,5,3,0,4,2.85,1,12,24,13006
70748,4193.0,34452.0,3,27463.0,0,1,0,1,0,3,2,0,1,4.29,0,7,5,13006
70749,4193.0,34452.0,3,27463.0,0,1,0,1,0,2,2,0,1,5.18,1,10,20,13006


In [244]:
df_fresh_xgb_cat

,promotion_name,state_province,media_type,store_type,sales_region,country,member_card,education,occupation,gender,marital_status,yearly_income,houseowner
0,Sale_discount,WA,Cash Register Handout,Supermarket,North West,USA,Bronze,High School Degree,Skilled Manual,F,M,$30K - $50K,Y
1,Sale_discount,WA,Cash Register Handout,Supermarket,North West,USA,Bronze,High School Degree,Skilled Manual,F,M,$30K - $50K,Y
2,Sale_discount,WA,Cash Register Handout,Supermarket,North West,USA,Bronze,High School Degree,Skilled Manual,F,M,$30K - $50K,Y
3,Sale_discount,WA,Cash Register Handout,Supermarket,North West,USA,Normal,High School Degree,Skilled Manual,M,S,$50K - $70K,Y
4,Sale_discount,WA,Cash Register Handout,Supermarket,North West,USA,Normal,High School Degree,Skilled Manual,M,S,$50K - $70K,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,Sale_discount,BC,In-Store Coupon,Mid-Size Grocery,Canada West,Canada,Silver,High School Degree,Management,M,S,$110K - $130K,Y
70747,Sale_discount,BC,In-Store Coupon,Mid-Size Grocery,Canada West,Canada,Silver,High School Degree,Management,M,S,$110K - $130K,Y
70748,Sale_discount,BC,In-Store Coupon,Mid-Size Grocery,Canada West,Canada,Golden,Bachelors Degree,Professional,M,S,$90K - $110K,Y
70749,Sale_discount,BC,In-Store Coupon,Mid-Size Grocery,Canada West,Canada,Golden,Bachelors Degree,Professional,M,S,$90K - $110K,Y


In [245]:
for col in df_fresh_xgb_cat.columns:
    df_fresh_xgb_cat[col] = le.fit_transform(df_fresh_xgb_cat[col])

In [246]:
df_fresh_xgb_cat

,promotion_name,state_province,media_type,store_type,sales_region,country,member_card,education,occupation,gender,marital_status,yearly_income,houseowner
0,2,9,1,4,5,2,0,2,4,0,0,4,1
1,2,9,1,4,5,2,0,2,4,0,0,4,1
2,2,9,1,4,5,2,0,2,4,0,0,4,1
3,2,9,1,4,5,2,2,2,4,1,1,5,1
4,2,9,1,4,5,2,2,2,4,1,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,2,0,5,2,0,0,3,2,1,1,1,1,1
70747,2,0,5,2,0,0,3,2,1,1,1,1,1
70748,2,0,5,2,0,0,1,0,3,1,1,7,1
70749,2,0,5,2,0,0,1,0,3,1,1,7,1


In [247]:
df_fresh_xgb=pd.concat([df_fresh_xgb_cat,df_fresh_xgb_num],axis=1)

In [248]:
#Setting target variable
X = df_fresh_xgb.drop(columns=['cost'])
y = df_fresh_xgb['cost']

In [249]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression

In [250]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [251]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     72.07
Date:                Sun, 25 Feb 2024   Prob (F-statistic):               0.00
Time:                        23:35:42   Log-Likelihood:            -5.3185e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56569   BIC:                         1.064e+06
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.356e+04    209.557     64.717      0.000    1.32e+04     1.4e+04
promotion_name         -93.2501     12.973     -7.188      0.000    -118.677     -67.824
state_province         -34.7821      4.626     -7.518      0.000     -43.850     -25.714
media_type             -24.3121      3.611     -6.733      0.000     -31.390     -17.234
store_type            -239.0206     22.506    -10.620      0.000    -283.133    -194.908
sales_region          -145.5253     23.561     -6.177      0.000    -191.704     -99.346
country                 34.7746     65.117      0.534      0.593     -92.855     162.405
member_card              8.0041     13.637      0.587      0.557     -18.725      34.733
education               -4.9631      9.852     -0.504      0.614     -24.273      14.347
occupation              -3.2260     11.693     -0.276      0.783     -26.144      19.692
gender                 -38.9062     24.651     -1.578      0.115     -87.223       9.411
marital_status          16.9206     34.051      0.497      0.619     -49.820      83.662
yearly_income           -5.2996      7.945     -0.667      0.505     -20.872      10.273
houseowner              48.1615     26.222      1.837      0.066      -3.234      99.557
frozen_sqft              0.3673      0.166      2.210      0.027       0.042       0.693
store_sqft              -0.2212      0.100     -2.217      0.027      -0.417      -0.026
promotion_period       -68.6436      3.304    -20.773      0.000     -75.120     -62.167
grocery_sqft             0.1660      0.098      1.693      0.090      -0.026       0.358
video_store           -842.3398     47.810    -17.618      0.000    -936.048    -748.632
coffee_bar            -282.7240     54.165     -5.220      0.000    -388.888    -176.560
salad_bar              180.2961    126.739      1.423      0.155     -68.113     428.705
florist               -599.2676     60.629     -9.884      0.000    -718.101    -480.434
prepared_food         -206.1817    121.699     -1.694      0.090    -444.712      32.349
unit_sales             -10.9689     17.432     -0.629      0.529     -45.136      23.199
num_cars_owned          46.9954     12.951      3.629      0.000      21.612      72.379
num_children_at_home   -24.2538     14.163     -1.712      0.087     -52.013       3.505
total_children         -20.7840      9.477     -2.193      0.028     -39.359      -2.209
store_sales              2.6497      4.087      0.648      0.517      -5.361      10.660
recyclable_package       0.8857     24.721      0.036      0.971     -47.569      49.340
cases_per_pallet         2.4323      4.664      0.522      0.602      -6.709      11.574
units_per_case           0.6318      1.196      0.528      0.597      -1.713       2.976
=========================================================

In [252]:
significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const               0.000000e+00
promotion_name      6.644532e-13
state_province      5.640364e-14
media_type          1.682310e-11
store_type          2.543493e-26
sales_region        6.593900e-10
frozen_sqft         2.709493e-02
store_sqft          2.662660e-02
promotion_period    1.723344e-95
video_store         2.723812e-69
coffee_bar          1.798792e-07
florist             5.091699e-23
num_cars_owned      2.849490e-04
total_children      2.830824e-02
dtype: float64


In [253]:
sig_pvalues_features=significant_pvalues.index

In [254]:
sig_pvalues_features.drop("const")

Index(['promotion_name', 'state_province', 'media_type', 'store_type',
       'sales_region', 'frozen_sqft', 'store_sqft', 'promotion_period',
       'video_store', 'coffee_bar', 'florist', 'num_cars_owned',
       'total_children'],
      dtype='object')

In [255]:
len(significant_pvalues)

14

In [256]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [257]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8497856.836328592
Mean squared error for test data: 8499283.257660814


In [258]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [9332608.8580864  9690069.58466006 8942168.12276199]


In [259]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8942168.122761987 9690069.584660063 9321615.52183615


In [260]:
lr=LinearRegression()
model_sfs=sfs(estimator=lr,k_features="best",forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('promotion_name',
 'state_province',
 'media_type',
 'store_type',
 'sales_region',
 'gender',
 'marital_status',
 'houseowner',
 'frozen_sqft',
 'store_sqft',
 'promotion_period',
 'grocery_sqft',
 'video_store',
 'coffee_bar',
 'salad_bar',
 'florist',
 'prepared_food',
 'num_cars_owned',
 'total_children')

In [262]:
len(model_sfs.k_feature_names_)

19

# Decision Tree Regressor

In [263]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [264]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.299428
15,promotion_period,0.159476
0,promotion_name,0.145074
13,frozen_sqft,0.064043
16,grocery_sqft,0.058466
14,store_sqft,0.044909
1,state_province,0.044843
3,store_type,0.032836
20,florist,0.030426
21,prepared_food,0.025713


In [265]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [266]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 0.0
Mean squared error for test data: 367940.0023319907


In [267]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [13094632.03731343 15700552.70768318 12832129.57655939]


In [268]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

12832129.576559385 15700552.707683176 13875771.440518664


# Random Forest Regressor

In [269]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [270]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.310498
15,promotion_period,0.207484
0,promotion_name,0.093024
13,frozen_sqft,0.066460
16,grocery_sqft,0.061371
1,state_province,0.049983
14,store_sqft,0.046417
3,store_type,0.028511
4,sales_region,0.018853
18,coffee_bar,0.013734


In [271]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [272]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 29898.50078119258
Mean squared error for test data: 233115.9767027984


In [273]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10384908.32431008 12210905.50743598 11739939.84784211]


In [274]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10384908.32431008 12210905.507435983 11445251.22652939


# Ada Boost Regressor

In [275]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [276]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.359308
15,promotion_period,0.280123
4,sales_region,0.096897
1,state_province,0.086330
3,store_type,0.064592
16,grocery_sqft,0.038300
14,store_sqft,0.033604
13,frozen_sqft,0.020820
0,promotion_name,0.011243
17,video_store,0.008783


In [277]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [278]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 7866004.848954066
Mean squared error for test data: 7893522.383171979


In [279]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 8785694.6893474  10352301.85061023  8950757.56052362]


In [280]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8785694.689347396 10352301.850610228 9362918.033493748


# Gradient Boost Regressor

In [281]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [282]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.284431
15,promotion_period,0.157686
0,promotion_name,0.131568
16,grocery_sqft,0.110536
14,store_sqft,0.068118
13,frozen_sqft,0.057083
1,state_province,0.050210
3,store_type,0.043580
4,sales_region,0.031872
18,coffee_bar,0.026728


In [283]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [284]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5161114.369448774
Mean squared error for test data: 5229404.992687927


In [285]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9992398.94993131 11022770.96213338 11115035.54304625]


In [286]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9992398.949931314 11115035.543046245 10710068.485036978


# XGBoost Regressor

In [287]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [288]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
0,promotion_name,0.118133
13,frozen_sqft,0.103182
16,grocery_sqft,0.092843
15,promotion_period,0.090805
18,coffee_bar,0.081063
2,media_type,0.079605
14,store_sqft,0.076536
17,video_store,0.072273
4,sales_region,0.064106
1,state_province,0.059177


In [289]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [290]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 415855.8003192052
Mean squared error for test data: 518338.9939876872


In [291]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9914806.39688347 12988717.72982182 12695941.38381255]


In [292]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9914806.396883465 12988717.729821822 11866488.503505947


# Stacking Regressor

In [293]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [294]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [295]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 227321.86259717314
Mean squared error for test data: 458862.054837114


In [296]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [18186834.17999491 16787630.33306479 18613236.25166433]


In [297]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

16787630.33306479 18613236.251664333 17862566.92157468


In [298]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [299]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [300]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 17176.42328265528
Mean squared error for test data: 254143.694845392


In [301]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10984143.99104407 14294085.09436911 13208211.47479875]


In [302]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10984143.991044067 14294085.094369106 12828813.520070642


# selecting 16 columns based on domain

In [304]:
l=['department', 'promotion_name', 'media_type', 'sales_region', 
'store_type', 'video_store', 'yearly_income', 'member_card', 'occupation', 
'store_cost', 'unit_sales', 'SRP', 'net_weight', 'cost', 'store_sqft', 
'promotion_period']

In [503]:
df_last=df1[['department', 'promotion_name', 'media_type', 'sales_region', 
'store_type', 'video_store', 'yearly_income', 'member_card', 'occupation', 
'store_cost', 'unit_sales', 'SRP', 'net_weight', 'cost', 'store_sqft', 
'promotion_period']]

In [504]:
df_last['promotion_name']=df_last['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\3094112243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last['promotion_name']=df_last['promotion_name'].apply(func_prom_name)


In [505]:
df_last["department"] = df_last["department"].apply(func_dept)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\2225853781.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last["department"] = df_last["department"].apply(func_dept)


In [506]:
df_last

,department,promotion_name,media_type,sales_region,store_type,video_store,yearly_income,member_card,occupation,store_cost,unit_sales,SRP,net_weight,cost,store_sqft,promotion_period
0,Household,Sale_discount,Cash Register Handout,North West,Supermarket,0,$30K - $50K,Bronze,Skilled Manual,2.2784,4,1.78,11.30,12950,39696.0,4
1,Dairy,Sale_discount,Cash Register Handout,North West,Supermarket,0,$30K - $50K,Bronze,Skilled Manual,1.7856,4,1.44,5.25,12950,39696.0,4
2,Health and Hygiene,Sale_discount,Cash Register Handout,North West,Supermarket,0,$30K - $50K,Bronze,Skilled Manual,2.0535,3,1.85,4.11,12950,39696.0,4
3,Produce,Sale_discount,Cash Register Handout,North West,Supermarket,0,$50K - $70K,Normal,Skilled Manual,2.2440,2,3.74,12.10,12950,39696.0,4
4,Ready_to_eat,Sale_discount,Cash Register Handout,North West,Supermarket,0,$50K - $70K,Normal,Skilled Manual,1.0488,4,0.57,14.80,12950,39696.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,0,$110K - $130K,Silver,Management,3.5328,3,2.56,8.50,13006,34452.0,3
70747,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,0,$110K - $130K,Silver,Management,0.8835,5,0.57,7.30,13006,34452.0,3
70748,Household,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,0,$90K - $110K,Golden,Professional,1.4586,3,1.43,7.19,13006,34452.0,3
70749,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,0,$90K - $110K,Golden,Professional,2.0720,2,2.59,7.77,13006,34452.0,3


In [507]:
le=LabelEncoder()
df_last_num=df_last.select_dtypes(include=np.number)
df_last_cat=df_last.select_dtypes(exclude=np.number)

In [508]:
df_last_num

,video_store,store_cost,unit_sales,SRP,net_weight,cost,store_sqft,promotion_period
0,0,2.2784,4,1.78,11.30,12950,39696.0,4
1,0,1.7856,4,1.44,5.25,12950,39696.0,4
2,0,2.0535,3,1.85,4.11,12950,39696.0,4
3,0,2.2440,2,3.74,12.10,12950,39696.0,4
4,0,1.0488,4,0.57,14.80,12950,39696.0,4
...,...,...,...,...,...,...,...,...
70746,0,3.5328,3,2.56,8.50,13006,34452.0,3
70747,0,0.8835,5,0.57,7.30,13006,34452.0,3
70748,0,1.4586,3,1.43,7.19,13006,34452.0,3
70749,0,2.0720,2,2.59,7.77,13006,34452.0,3


In [509]:
df_last_cat

,department,promotion_name,media_type,sales_region,store_type,yearly_income,member_card,occupation
0,Household,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,Bronze,Skilled Manual
1,Dairy,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,Bronze,Skilled Manual
2,Health and Hygiene,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,Bronze,Skilled Manual
3,Produce,Sale_discount,Cash Register Handout,North West,Supermarket,$50K - $70K,Normal,Skilled Manual
4,Ready_to_eat,Sale_discount,Cash Register Handout,North West,Supermarket,$50K - $70K,Normal,Skilled Manual
...,...,...,...,...,...,...,...,...
70746,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$110K - $130K,Silver,Management
70747,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$110K - $130K,Silver,Management
70748,Household,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$90K - $110K,Golden,Professional
70749,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$90K - $110K,Golden,Professional


In [510]:
for col in df_last_cat.columns:
    df_last_cat[col] = le.fit_transform(df_last_cat[col])

In [511]:
df_last_cat

,department,promotion_name,media_type,sales_region,store_type,yearly_income,member_card,occupation
0,7,2,1,5,4,4,0,4
1,4,2,1,5,4,4,0,4
2,6,2,1,5,4,4,0,4
3,10,2,1,5,4,5,2,4
4,11,2,1,5,4,5,2,4
...,...,...,...,...,...,...,...,...
70746,10,2,5,0,2,1,3,1
70747,10,2,5,0,2,1,3,1
70748,7,2,5,0,2,7,1,3
70749,10,2,5,0,2,7,1,3


In [512]:
df_last=pd.concat([df_last_cat,df_last_num],axis=1)

In [513]:
df_last

,department,promotion_name,media_type,sales_region,store_type,yearly_income,member_card,occupation,video_store,store_cost,unit_sales,SRP,net_weight,cost,store_sqft,promotion_period
0,7,2,1,5,4,4,0,4,0,2.2784,4,1.78,11.30,12950,39696.0,4
1,4,2,1,5,4,4,0,4,0,1.7856,4,1.44,5.25,12950,39696.0,4
2,6,2,1,5,4,4,0,4,0,2.0535,3,1.85,4.11,12950,39696.0,4
3,10,2,1,5,4,5,2,4,0,2.2440,2,3.74,12.10,12950,39696.0,4
4,11,2,1,5,4,5,2,4,0,1.0488,4,0.57,14.80,12950,39696.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,10,2,5,0,2,1,3,1,0,3.5328,3,2.56,8.50,13006,34452.0,3
70747,10,2,5,0,2,1,3,1,0,0.8835,5,0.57,7.30,13006,34452.0,3
70748,7,2,5,0,2,7,1,3,0,1.4586,3,1.43,7.19,13006,34452.0,3
70749,10,2,5,0,2,7,1,3,0,2.0720,2,2.59,7.77,13006,34452.0,3


In [514]:
#Setting target variable
X = df_last.drop(columns=['cost'])
y = df_last['cost']

In [515]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression

In [516]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [517]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     121.3
Date:                Mon, 26 Feb 2024   Prob (F-statistic):               0.00
Time:                        01:28:42   Log-Likelihood:            -5.3201e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56584   BIC:                         1.064e+06
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.247e+04    133.884     93.103      0.000    1.22e+04    1.27e+04
department          -1.2897      3.089     -0.418      0.676      -7.344       4.764
promotion_name     -71.7283     12.877     -5.570      0.000     -96.968     -46.488
media_type         -24.5991      3.588     -6.856      0.000     -31.632     -17.566
sales_region      -154.1774      7.971    -19.342      0.000    -169.801    -138.554
store_type         -49.3852     10.214     -4.835      0.000     -69.404     -29.366
yearly_income        2.4188      6.475      0.374      0.709     -10.271      15.109
member_card        -10.7153     13.361     -0.802      0.423     -36.903      15.472
occupation          -3.7872     11.682     -0.324      0.746     -26.684      19.110
video_store       -833.6702     38.321    -21.755      0.000    -908.779    -758.561
store_cost          30.4400     22.896      1.329      0.184     -14.436      75.316
unit_sales         -50.4666     24.404     -2.068      0.039     -98.298      -2.636
SRP                -30.1860     31.306     -0.964      0.335     -91.546      31.174
net_weight          -1.3327      2.637     -0.505      0.613      -6.501       3.836
store_sqft          -0.0310      0.002    -12.525      0.000      -0.036      -0.026
promotion_period   -65.3985      3.260    -20.062      0.000     -71.788     -59.009
==============================================================================
Omnibus:                    42376.037   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3313.279
Skew:                           0.050   Prob(JB):                         0.00
Kurtosis:                       1.819   Cond. No.                     3.16e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [518]:
significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const                0.000000e+00
promotion_name       2.557784e-08
media_type           7.177605e-12
sales_region         4.441977e-83
store_type           1.333489e-06
video_store         1.661398e-104
unit_sales           3.864418e-02
store_sqft           6.117111e-36
promotion_period     3.216906e-89
dtype: float64


In [519]:
sig_pvalues_features=significant_pvalues.index

In [520]:
sig_pvalues_features.drop("const")

Index(['promotion_name', 'media_type', 'sales_region', 'store_type',
       'video_store', 'unit_sales', 'store_sqft', 'promotion_period'],
      dtype='object')

In [521]:
len(significant_pvalues)

9

In [522]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [523]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8547702.924184188
Mean squared error for test data: 8547936.069266034


In [524]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [9018370.5279005  9543997.2919886  8725689.57926479]


In [525]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8725689.579264792 9543997.291988598 9096019.133051297


In [526]:
lr=LinearRegression()
model_sfs=sfs(estimator=lr,k_features="best",forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'yearly_income',
 'video_store',
 'store_cost',
 'unit_sales',
 'store_sqft',
 'promotion_period')

In [527]:
len(model_sfs.k_feature_names_)

10

# Decision Tree Regressor

In [528]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [529]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.324397
14,promotion_period,0.189230
13,store_sqft,0.142100
1,promotion_name,0.120352
4,store_type,0.083143
3,sales_region,0.061862
8,video_store,0.024988
9,store_cost,0.011857
12,net_weight,0.008907
5,yearly_income,0.008625


In [530]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [531]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 3.4397173144876323
Mean squared error for test data: 802214.8378913151


In [532]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [14145265.84832938 15416397.01110923 11640174.38430225]


In [533]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

11640174.384302251 15416397.011109227 13733945.74791362


# Random Forest Regressor

In [534]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [535]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.316465
14,promotion_period,0.205249
13,store_sqft,0.180983
1,promotion_name,0.094362
3,sales_region,0.060345
4,store_type,0.056601
8,video_store,0.031269
9,store_cost,0.010806
12,net_weight,0.010562
11,SRP,0.009583


In [536]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [537]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 61438.996493415834
Mean squared error for test data: 475076.8008482782


In [538]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [11413542.13998184 11982262.62276192 11443690.11041707]


In [539]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

11413542.139981838 11982262.622761924 11613164.957720278


# Ada Boost Regressor

In [540]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [541]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,0.393494
14,promotion_period,0.256532
3,sales_region,0.148021
13,store_sqft,0.086530
1,promotion_name,0.068726
4,store_type,0.030483
8,video_store,0.016213
0,department,0.000000
5,yearly_income,0.000000
6,member_card,0.000000


In [542]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [543]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8043919.3562889
Mean squared error for test data: 8076911.360869022


In [544]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [8986130.03918351 9873057.85971718 9056238.93610436]


In [545]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8986130.039183512 9873057.859717181 9305142.27833502


# Gradient Boost Regressor

In [546]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [547]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
2,media_type,2.807894e-01
13,store_sqft,2.158251e-01
14,promotion_period,2.044608e-01
1,promotion_name,1.400617e-01
4,store_type,7.167617e-02
3,sales_region,6.648259e-02
8,video_store,2.070431e-02
9,store_cost,5.542124e-20
0,department,0.000000e+00
5,yearly_income,0.000000e+00


In [548]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [549]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5325194.78499707
Mean squared error for test data: 5359234.322814723


In [550]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9788594.89013384 11309296.60692428 10142530.55044575]


In [551]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9788594.890133841 11309296.606924282 10413474.015834624


# XGBoost Regressor

In [552]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [553]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
14,promotion_period,0.184789
8,video_store,0.168397
1,promotion_name,0.153429
13,store_sqft,0.139599
2,media_type,0.127299
4,store_type,0.108478
3,sales_region,0.088459
10,unit_sales,0.007471
6,member_card,0.005197
7,occupation,0.003585


In [554]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [555]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 507923.3102938993
Mean squared error for test data: 611269.8076461137


In [556]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10138471.85211087 12377013.30200347 12990842.5372271 ]


In [557]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10138471.852110868 12990842.537227102 11835442.56378048


# Stacking Regressor

In [558]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [559]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [560]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 581464.946360424
Mean squared error for test data: 891388.3834357996


In [561]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [16514026.99330054 15793441.92388908 12900979.78755883]


In [562]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

12900979.787558835 16514026.993300542 15069482.901582817


In [563]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [564]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [565]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 134841.36995060186
Mean squared error for test data: 541272.7176894931


In [566]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10708767.4903938  13234500.92202379 12028354.43145904]


In [567]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10708767.4903938 13234500.922023788 11990540.947958877


# selecting 16 columns based on sfs

In [568]:
df_last_sfs=df1[['promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'marital_status',
 'gender',
 'education',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'prepared_food',
 'florist',
 'grocery_sqft',
 'total_children',
 'promotion_period',"cost"]]

In [569]:
df_last_sfs['promotion_name']=df_last_sfs['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\1843409717.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last_sfs['promotion_name']=df_last_sfs['promotion_name'].apply(func_prom_name)


In [570]:
le=LabelEncoder()
df_last_sfs_num=df_last_sfs.select_dtypes(include=np.number)
df_last_sfs_cat=df_last_sfs.select_dtypes(exclude=np.number)

In [571]:
df_last_sfs_num

,coffee_bar,video_store,salad_bar,prepared_food,florist,grocery_sqft,total_children,promotion_period,cost
0,0,0,1,1,0,24390.0,2,4,12950
1,0,0,1,1,0,24390.0,2,4,12950
2,0,0,1,1,0,24390.0,2,4,12950
3,0,0,1,1,0,24390.0,2,4,12950
4,0,0,1,1,0,24390.0,2,4,12950
...,...,...,...,...,...,...,...,...,...
70746,1,0,0,0,1,27463.0,4,3,13006
70747,1,0,0,0,1,27463.0,4,3,13006
70748,1,0,0,0,1,27463.0,1,3,13006
70749,1,0,0,0,1,27463.0,1,3,13006


In [572]:
df_last_sfs_cat

,promotion_name,media_type,sales_region,store_type,marital_status,gender,education
0,Sale_discount,Cash Register Handout,North West,Supermarket,M,F,High School Degree
1,Sale_discount,Cash Register Handout,North West,Supermarket,M,F,High School Degree
2,Sale_discount,Cash Register Handout,North West,Supermarket,M,F,High School Degree
3,Sale_discount,Cash Register Handout,North West,Supermarket,S,M,High School Degree
4,Sale_discount,Cash Register Handout,North West,Supermarket,S,M,High School Degree
...,...,...,...,...,...,...,...
70746,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,S,M,High School Degree
70747,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,S,M,High School Degree
70748,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,S,M,Bachelors Degree
70749,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,S,M,Bachelors Degree


In [573]:
for col in df_last_sfs_cat.columns:
    df_last_sfs_cat[col] = le.fit_transform(df_last_sfs_cat[col])

In [574]:
df_last_sfs=pd.concat([df_last_sfs_cat,df_last_sfs_num],axis=1)

In [575]:
#Setting target variable
X = df_last_sfs.drop(columns=['cost'])
y = df_last_sfs['cost']

In [576]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression

In [577]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [578]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     138.6
Date:                Mon, 26 Feb 2024   Prob (F-statistic):               0.00
Time:                        01:44:52   Log-Likelihood:            -5.3189e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56584   BIC:                         1.064e+06
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.363e+04    154.449     88.257      0.000    1.33e+04    1.39e+04
promotion_name     -83.6885     12.883     -6.496      0.000    -108.939     -58.438
media_type         -25.3935      3.586     -7.081      0.000     -32.423     -18.364
sales_region      -153.1963      8.494    -18.036      0.000    -169.845    -136.548
store_type        -202.8596     18.002    -11.269      0.000    -238.144    -167.575
marital_status      36.3981     24.574      1.481      0.139     -11.768      84.564
gender             -28.1101     24.599     -1.143      0.253     -76.324      20.103
education          -10.4129      7.934     -1.312      0.189     -25.964       5.138
coffee_bar        -352.5305     46.839     -7.526      0.000    -444.336    -260.725
video_store       -777.6452     39.474    -19.700      0.000    -855.014    -700.276
salad_bar          173.2169     85.410      2.028      0.043       5.812     340.622
prepared_food     -216.7640     79.832     -2.715      0.007    -373.236     -60.292
florist           -438.1009     51.563     -8.496      0.000    -539.165    -337.037
grocery_sqft        -0.0684      0.005    -13.987      0.000      -0.078      -0.059
total_children     -26.5069      8.228     -3.221      0.001     -42.634     -10.380
promotion_period   -67.0624      3.262    -20.561      0.000     -73.455     -60.669
==============================================================================
Omnibus:                    35977.743   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3207.116
Skew:                           0.055   Prob(JB):                         0.00
Kurtosis:                       1.839   Cond. No.                     2.84e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.84e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [579]:
significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const               0.000000e+00
promotion_name      8.317368e-11
media_type          1.451106e-12
sales_region        1.632651e-72
store_type          2.013872e-29
coffee_bar          5.294626e-14
video_store         4.180302e-86
salad_bar           4.255929e-02
prepared_food       6.624724e-03
florist             2.002017e-17
grocery_sqft        2.218766e-44
total_children      1.276129e-03
promotion_period    1.361814e-93
dtype: float64


In [580]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [581]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8509918.656533688
Mean squared error for test data: 8513280.24846813


In [582]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [9268524.67156538 9512596.27113023 8881827.37643359]


In [583]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8881827.37643359 9512596.271130228 9220982.773043066


# Decision Tree Regressor

In [584]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [585]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,media_type,0.332390
14,promotion_period,0.173809
12,grocery_sqft,0.154104
0,promotion_name,0.130781
3,store_type,0.051048
11,florist,0.037608
10,prepared_food,0.025339
2,sales_region,0.023986
7,coffee_bar,0.020695
13,total_children,0.012834


In [586]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [587]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 202784.25610043146
Mean squared error for test data: 277484.37198107375


In [588]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [14001316.76439129 15462199.52278369 12866502.42582942]


In [589]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

12866502.42582942 15462199.522783691 14110006.237668132


# Random Forest Regressor

In [590]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [591]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,media_type,0.300489
14,promotion_period,0.214121
12,grocery_sqft,0.157698
0,promotion_name,0.105012
2,sales_region,0.050212
3,store_type,0.044613
8,video_store,0.023663
11,florist,0.023171
7,coffee_bar,0.019928
10,prepared_food,0.016149


In [592]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [593]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 204580.16080800563
Mean squared error for test data: 276844.5202987722


In [594]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10271784.42367941 12639341.18282837 11811577.5076396 ]


In [595]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10271784.42367941 12639341.182828367 11574234.37138246


# Ada Boost Regressor

In [596]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [597]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,media_type,0.354890
14,promotion_period,0.242602
12,grocery_sqft,0.177409
2,sales_region,0.147187
0,promotion_name,0.041890
3,store_type,0.029501
9,salad_bar,0.006521
4,marital_status,0.000000
5,gender,0.000000
6,education,0.000000


In [598]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [599]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 7964453.648771122
Mean squared error for test data: 8015538.263305666


In [600]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [8704698.6527967  9863482.3266044  8911165.83907301]


In [601]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8704698.652796702 9863482.3266044 9159782.272824705


# Gradient Boost Regressor

In [602]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [603]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,media_type,0.309483
14,promotion_period,0.190736
12,grocery_sqft,0.186290
0,promotion_name,0.132417
2,sales_region,0.069349
3,store_type,0.044980
11,florist,0.020239
8,video_store,0.017143
7,coffee_bar,0.017089
10,prepared_food,0.008562


In [604]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [605]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5188831.807173309
Mean squared error for test data: 5221569.0343340235


In [606]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9444272.29557126 11490377.02706782 10896432.16611255]


In [607]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9444272.295571262 11490377.027067816 10610360.496250542


# XGBoost Regressor

In [608]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [609]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
11,florist,0.146435
8,video_store,0.140200
0,promotion_name,0.120853
14,promotion_period,0.097946
7,coffee_bar,0.096869
12,grocery_sqft,0.093254
1,media_type,0.085973
2,sales_region,0.070256
10,prepared_food,0.057955
3,store_type,0.055954


In [610]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [611]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 483229.7606107851
Mean squared error for test data: 522626.7413899384


In [612]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9727425.87093152 13473378.92560242 13306642.14616497]


In [613]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9727425.870931525 13473378.925602416 12169148.980899638


# Stacking Regressor

In [614]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [615]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [616]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 319071.7568675387
Mean squared error for test data: 385841.2301443533


In [617]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [18306032.3359667  16027801.19807964 13241372.90482252]


In [618]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

13241372.904822523 18306032.335966703 15858402.146289619


In [619]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [620]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [621]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 213479.31056329937
Mean squared error for test data: 278958.7061991039


In [622]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [13931176.35056266 11622641.9484106  11376621.77444605]


In [623]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

11376621.774446053 13931176.350562664 12310146.691139773


# selecting 16 columns using rel imp of xgb model for 31 columns

In [624]:
df_last_xgb=df1[['video_store', 'promotion_period', 'promotion_name', 'store_sqft',
       'grocery_sqft', 'salad_bar', 'media_type', 'coffee_bar',
       'store_type', 'sales_region', 'florist', 'prepared_food',
       'country', 'marital_status', 'member_card',
        "cost"]]

In [625]:
df_last_xgb['promotion_name']=df_last_xgb['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\731837663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last_xgb['promotion_name']=df_last_xgb['promotion_name'].apply(func_prom_name)


In [626]:
le=LabelEncoder()
df_last_xgb_num=df_last_xgb.select_dtypes(include=np.number)
df_last_xgb_cat=df_last_xgb.select_dtypes(exclude=np.number)

In [627]:
df_last_xgb_num

,video_store,promotion_period,store_sqft,grocery_sqft,salad_bar,coffee_bar,florist,prepared_food,cost
0,0,4,39696.0,24390.0,1,0,0,1,12950
1,0,4,39696.0,24390.0,1,0,0,1,12950
2,0,4,39696.0,24390.0,1,0,0,1,12950
3,0,4,39696.0,24390.0,1,0,0,1,12950
4,0,4,39696.0,24390.0,1,0,0,1,12950
...,...,...,...,...,...,...,...,...,...
70746,0,3,34452.0,27463.0,0,1,1,0,13006
70747,0,3,34452.0,27463.0,0,1,1,0,13006
70748,0,3,34452.0,27463.0,0,1,1,0,13006
70749,0,3,34452.0,27463.0,0,1,1,0,13006


In [628]:
df_last_xgb_cat

,promotion_name,media_type,store_type,sales_region,country,marital_status,member_card
0,Sale_discount,Cash Register Handout,Supermarket,North West,USA,M,Bronze
1,Sale_discount,Cash Register Handout,Supermarket,North West,USA,M,Bronze
2,Sale_discount,Cash Register Handout,Supermarket,North West,USA,M,Bronze
3,Sale_discount,Cash Register Handout,Supermarket,North West,USA,S,Normal
4,Sale_discount,Cash Register Handout,Supermarket,North West,USA,S,Normal
...,...,...,...,...,...,...,...
70746,Sale_discount,In-Store Coupon,Mid-Size Grocery,Canada West,Canada,S,Silver
70747,Sale_discount,In-Store Coupon,Mid-Size Grocery,Canada West,Canada,S,Silver
70748,Sale_discount,In-Store Coupon,Mid-Size Grocery,Canada West,Canada,S,Golden
70749,Sale_discount,In-Store Coupon,Mid-Size Grocery,Canada West,Canada,S,Golden


In [629]:
for col in df_last_xgb_cat.columns:
    df_last_xgb_cat[col] = le.fit_transform(df_last_xgb_cat[col])

In [630]:
df_last_xgb=pd.concat([df_last_xgb_cat,df_last_xgb_num],axis=1)

In [631]:
#Setting target variable
X = df_last_xgb.drop(columns=['cost'])
y = df_last_xgb['cost']

In [632]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression

In [633]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [634]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     137.8
Date:                Mon, 26 Feb 2024   Prob (F-statistic):               0.00
Time:                        01:54:33   Log-Likelihood:            -5.3189e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56584   BIC:                         1.064e+06
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.352e+04    152.594     88.584      0.000    1.32e+04    1.38e+04
promotion_name     -83.6767     12.904     -6.485      0.000    -108.968     -58.386
media_type         -25.2769      3.589     -7.042      0.000     -32.312     -18.242
store_type        -199.3215     18.212    -10.945      0.000    -235.016    -163.627
sales_region      -156.9432     23.455     -6.691      0.000    -202.915    -110.971
country              4.6112     64.810      0.071      0.943    -122.416     131.638
marital_status      37.5970     24.619      1.527      0.127     -10.656      85.850
member_card         -7.0296     11.685     -0.602      0.547     -29.933      15.874
video_store       -770.9775     42.473    -18.152      0.000    -854.225    -687.730
promotion_period   -66.9608      3.298    -20.302      0.000     -73.425     -60.496
store_sqft           0.0041      0.007      0.618      0.536      -0.009       0.017
grocery_sqft        -0.0738      0.010     -7.542      0.000      -0.093      -0.055
salad_bar          165.6365     85.805      1.930      0.054      -2.543     333.816
coffee_bar        -350.8735     48.009     -7.309      0.000    -444.971    -256.776
florist           -434.8266     54.751     -7.942      0.000    -542.139    -327.514
prepared_food     -212.1453     81.596     -2.600      0.009    -372.074     -52.216
==============================================================================
Omnibus:                    36122.254   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3210.729
Skew:                           0.056   Prob(JB):                         0.00
Kurtosis:                       1.839   Cond. No.                     4.99e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.99e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [635]:
significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const               0.000000e+00
promotion_name      8.960655e-11
media_type          1.913029e-12
store_type          7.504499e-28
sales_region        2.233832e-11
video_store         1.995153e-73
promotion_period    2.611140e-91
grocery_sqft        4.683681e-14
coffee_bar          2.736330e-13
florist             2.028302e-15
prepared_food       9.326370e-03
dtype: float64


In [636]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [637]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8511786.009836433
Mean squared error for test data: 8512486.744388385


In [638]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [9256306.35063223 9694177.95599721 8918951.10254164]


In [639]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8918951.10254164 9694177.955997214 9289811.803057028


# Decision Tree Regressor

In [640]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [641]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,media_type,0.346944
8,promotion_period,0.189574
0,promotion_name,0.127473
10,grocery_sqft,0.105081
9,store_sqft,0.046919
2,store_type,0.046584
3,sales_region,0.035200
14,prepared_food,0.033559
13,florist,0.028541
12,coffee_bar,0.019106


In [642]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [643]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 454587.8717690571
Mean squared error for test data: 478692.9427899355


In [644]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [13338099.89982529 18334055.2061157  11582751.66748646]


In [645]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

11582751.66748646 18334055.2061157 14418302.25780915


# Random Forest Regressor

In [646]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [647]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,media_type,0.321588
8,promotion_period,0.220829
0,promotion_name,0.098933
10,grocery_sqft,0.098390
9,store_sqft,0.089117
2,store_type,0.039727
3,sales_region,0.034988
7,video_store,0.023304
12,coffee_bar,0.017125
13,florist,0.014894


In [648]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [649]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 454752.8822948093
Mean squared error for test data: 479063.04330998816


In [650]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10497779.13888552 13230399.51925337 11674357.92783569]


In [651]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10497779.138885522 13230399.519253373 11800845.528658196


# Ada Boost Regressor

In [652]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [653]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,media_type,0.367437
8,promotion_period,0.232559
10,grocery_sqft,0.161858
3,sales_region,0.112361
9,store_sqft,0.058471
4,country,0.036495
2,store_type,0.029713
0,promotion_name,0.001106
5,marital_status,0.000000
6,member_card,0.000000


In [654]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [655]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 7957667.719402413
Mean squared error for test data: 7996322.291602776


In [656]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 8704698.6527967  10019673.06986908  9011188.46382627]


In [657]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8704698.652796702 10019673.069869077 9245186.728830684


# Gradient Boost Regressor

In [658]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [659]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
1,media_type,2.895753e-01
8,promotion_period,1.707472e-01
10,grocery_sqft,1.555940e-01
0,promotion_name,1.199247e-01
9,store_sqft,1.085833e-01
2,store_type,4.942188e-02
3,sales_region,3.850093e-02
12,coffee_bar,2.513752e-02
7,video_store,1.714095e-02
14,prepared_food,9.060213e-03


In [660]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [661]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5111306.967585399
Mean squared error for test data: 5158887.688868468


In [662]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9634520.43998125 11992062.0568579  10629789.88225846]


In [663]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9634520.439981254 11992062.056857903 10752124.126365874


# XGB Regressor

In [664]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [665]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False).head(15)

,feature,importance
7,video_store,0.123442
8,promotion_period,0.100454
10,grocery_sqft,0.100008
0,promotion_name,0.096935
9,store_sqft,0.089193
1,media_type,0.077831
11,salad_bar,0.074497
3,sales_region,0.073056
13,florist,0.068747
12,coffee_bar,0.067504


In [666]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [667]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 498940.67255895276
Mean squared error for test data: 517326.77153206314


In [668]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9776369.73219024 12351626.72951936 12307378.32216272]


In [669]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9776369.732190236 12351626.729519363 11478458.261290774


# Stacking Regressor

In [670]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [671]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [672]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 619901.6259239891
Mean squared error for test data: 626656.6713121436


In [673]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [17478495.28695531 16271202.49273676 14601044.76430788]


In [674]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

14601044.764307879 17478495.286955312 16116914.181333318


In [675]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [676]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [677]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 465304.096080991
Mean squared error for test data: 496654.05687896133


In [678]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [13595264.91506285 10982918.63946572 10423721.84270242]


In [679]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10423721.842702422 13595264.915062852 11667301.799076999


# select 21 columns based on domain

In [ ]:
* keeping only 1 put of location related columns and a few of consumer related column while focusing on store related services and promotion related columns.

In [680]:
l=['department', 'promotion_name', 'media_type', 'sales_region', 
'store_type', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist', 
'yearly_income', 'member_card', 'occupation', 
'store_cost', 'unit_sales', 'cost', 'store_sqft', 'grocery_sqft', 'frozen_sqft', 'meat_sqft', 'promotion_period']

In [681]:
df_final=df1[['department', 'promotion_name', 'media_type', 'sales_region', 
'store_type', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist', 
'yearly_income', 'member_card', 'occupation', 
'store_cost', 'unit_sales', 'cost', 'store_sqft', 'grocery_sqft', 'frozen_sqft', 'meat_sqft', 'promotion_period']]

In [682]:
df_final['promotion_name']=df_final['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\82660395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['promotion_name']=df_final['promotion_name'].apply(func_prom_name)


In [683]:
df_final["department"] = df_final["department"].apply(func_dept)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\797592858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["department"] = df_final["department"].apply(func_dept)


In [684]:
le=LabelEncoder()
df_final_num=df_final.select_dtypes(include=np.number)
df_final_cat=df_final.select_dtypes(exclude=np.number)

In [685]:
df_final_num

,coffee_bar,video_store,salad_bar,prepared_food,florist,store_cost,unit_sales,cost,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,promotion_period
0,0,0,1,1,0,2.2784,4,12950,39696.0,24390.0,9184.0,6122.0,4
1,0,0,1,1,0,1.7856,4,12950,39696.0,24390.0,9184.0,6122.0,4
2,0,0,1,1,0,2.0535,3,12950,39696.0,24390.0,9184.0,6122.0,4
3,0,0,1,1,0,2.2440,2,12950,39696.0,24390.0,9184.0,6122.0,4
4,0,0,1,1,0,1.0488,4,12950,39696.0,24390.0,9184.0,6122.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,1,0,0,0,1,3.5328,3,13006,34452.0,27463.0,4193.0,2795.0,3
70747,1,0,0,0,1,0.8835,5,13006,34452.0,27463.0,4193.0,2795.0,3
70748,1,0,0,0,1,1.4586,3,13006,34452.0,27463.0,4193.0,2795.0,3
70749,1,0,0,0,1,2.0720,2,13006,34452.0,27463.0,4193.0,2795.0,3


In [686]:
df_final_cat

,department,promotion_name,media_type,sales_region,store_type,yearly_income,member_card,occupation
0,Household,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,Bronze,Skilled Manual
1,Dairy,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,Bronze,Skilled Manual
2,Health and Hygiene,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,Bronze,Skilled Manual
3,Produce,Sale_discount,Cash Register Handout,North West,Supermarket,$50K - $70K,Normal,Skilled Manual
4,Ready_to_eat,Sale_discount,Cash Register Handout,North West,Supermarket,$50K - $70K,Normal,Skilled Manual
...,...,...,...,...,...,...,...,...
70746,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$110K - $130K,Silver,Management
70747,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$110K - $130K,Silver,Management
70748,Household,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$90K - $110K,Golden,Professional
70749,Produce,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$90K - $110K,Golden,Professional


In [687]:
for col in df_final_cat.columns:
    df_final_cat[col] = le.fit_transform(df_final_cat[col])

In [688]:
df_final_cat

,department,promotion_name,media_type,sales_region,store_type,yearly_income,member_card,occupation
0,7,2,1,5,4,4,0,4
1,4,2,1,5,4,4,0,4
2,6,2,1,5,4,4,0,4
3,10,2,1,5,4,5,2,4
4,11,2,1,5,4,5,2,4
...,...,...,...,...,...,...,...,...
70746,10,2,5,0,2,1,3,1
70747,10,2,5,0,2,1,3,1
70748,7,2,5,0,2,7,1,3
70749,10,2,5,0,2,7,1,3


In [689]:
df_final=pd.concat([df_final_cat,df_final_num],axis=1)

In [690]:
#Setting target variable
X = df_final.drop(columns=['cost'])
y = df_final['cost']

In [691]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression

In [692]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [693]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     113.5
Date:                Mon, 26 Feb 2024   Prob (F-statistic):               0.00
Time:                        02:11:33   Log-Likelihood:            -5.3180e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56579   BIC:                         1.064e+06
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.586e+04    247.372     64.104      0.000    1.54e+04    1.63e+04
department          -1.2100      3.073     -0.394      0.694      -7.233       4.813
promotion_name     -71.2310     12.903     -5.520      0.000     -96.521     -45.941
media_type         -23.8681      3.599     -6.633      0.000     -30.921     -16.815
sales_region      -228.3654     10.577    -21.591      0.000    -249.096    -207.635
store_type        -437.1728     27.165    -16.093      0.000    -490.417    -383.929
yearly_income        4.4055      6.453      0.683      0.495      -8.242      17.053
member_card         -7.6154     13.329     -0.571      0.568     -33.740      18.509
occupation          -1.8240     11.651     -0.157      0.876     -24.661      21.013
coffee_bar        -702.6051     56.113    -12.521      0.000    -812.588    -592.622
video_store      -1145.6891     51.731    -22.147      0.000   -1247.082   -1044.296
salad_bar         1468.9261    150.431      9.765      0.000    1174.081    1763.772
prepared_food    -1539.8445    147.266    -10.456      0.000   -1828.486   -1251.203
florist           -886.4338     62.663    -14.146      0.000   -1009.254    -763.614
store_cost          10.1257      9.588      1.056      0.291      -8.667      28.918
unit_sales         -30.5664     16.917     -1.807      0.071     -63.724       2.591
store_sqft          -0.3905      0.101     -3.876      0.000      -0.588      -0.193
grocery_sqft         0.2812      0.098      2.858      0.004       0.088       0.474
frozen_sqft        502.7546     36.149     13.908      0.000     431.902     573.607
meat_sqft         -753.2007     54.182    -13.901      0.000    -859.398    -647.003
promotion_period   -58.6640      3.316    -17.689      0.000     -65.164     -52.164
==============================================================================
Omnibus:                    35566.306   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3195.832
Skew:                           0.052   Prob(JB):                         0.00
Kurtosis:                       1.841   Cond. No.                     9.14e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.14e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [694]:
significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const                0.000000e+00
promotion_name       3.395492e-08
media_type           3.329992e-11
sales_region        5.701071e-103
store_type           3.836536e-58
coffee_bar           6.383949e-36
video_store         3.217761e-108
salad_bar            1.661053e-22
prepared_food        1.447614e-25
florist              2.362497e-45
store_sqft           1.065149e-04
grocery_sqft         4.263995e-03
frozen_sqft          6.716363e-44
meat_sqft            7.352417e-44
promotion_period     7.908752e-70
dtype: float64


In [695]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [696]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8482289.197280211
Mean squared error for test data: 8487580.306828195


In [697]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [9751001.45769482 9676214.45618777 8957622.06285   ]


In [698]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8957622.062849998 9751001.45769482 9461612.658910863


# Decision Tree Regressor

In [699]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [700]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False)

,feature,importance
2,media_type,0.300104
19,promotion_period,0.174362
1,promotion_name,0.133426
18,meat_sqft,0.057335
17,frozen_sqft,0.054357
16,grocery_sqft,0.052879
15,store_sqft,0.036767
4,store_type,0.032376
8,coffee_bar,0.026056
13,store_cost,0.022595


In [701]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [702]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 3.4397173144876323
Mean squared error for test data: 728546.4673167974


In [703]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [14485520.06131275 18137361.66723202 13797861.68761396]


In [704]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

13797861.687613958 18137361.66723202 15473581.13871958


# Random Forest Regressor

In [705]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [706]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False)

,feature,importance
2,media_type,0.314494
19,promotion_period,0.209320
1,promotion_name,0.088983
16,grocery_sqft,0.056502
17,frozen_sqft,0.056116
18,meat_sqft,0.050372
15,store_sqft,0.049443
4,store_type,0.033442
3,sales_region,0.031354
13,store_cost,0.022277


In [707]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [708]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 60186.27658082431
Mean squared error for test data: 470051.844551537


In [709]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10504522.13365209 12167205.88468181 11777046.39076922]


In [710]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10504522.133652087 12167205.88468181 11482924.803034373


# Ada Boost Regressor

In [711]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [712]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False)

,feature,importance
2,media_type,0.365024
19,promotion_period,0.238387
3,sales_region,0.101515
16,grocery_sqft,0.086779
4,store_type,0.058122
17,frozen_sqft,0.052123
18,meat_sqft,0.039153
9,video_store,0.031426
15,store_sqft,0.027471
12,florist,0.000000


In [713]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [714]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 7930303.622046485
Mean squared error for test data: 7953035.242153933


In [715]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 8733995.90526298 10378052.76024162  9147555.94003414]


In [716]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8733995.905262984 10378052.760241624 9419868.201846251


# Gradient Boost Regressor

In [717]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [718]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False)

,feature,importance
2,media_type,2.915048e-01
19,promotion_period,1.513034e-01
1,promotion_name,1.297526e-01
16,grocery_sqft,1.219511e-01
15,store_sqft,5.978000e-02
18,meat_sqft,5.196077e-02
4,store_type,4.354314e-02
3,sales_region,4.219496e-02
17,frozen_sqft,3.969652e-02
8,coffee_bar,2.387783e-02


In [719]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [720]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5024162.9186538085
Mean squared error for test data: 5087735.59876688


In [721]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10153014.77283561 11144283.68627025 10946980.72149144]


In [722]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10153014.772835609 11144283.68627025 10748093.060199099


# XGBoost Regressor

In [723]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [724]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False)

,feature,importance
9,video_store,0.122378
17,frozen_sqft,0.116962
1,promotion_name,0.101626
15,store_sqft,0.098082
19,promotion_period,0.096458
8,coffee_bar,0.088246
16,grocery_sqft,0.085198
2,media_type,0.084298
4,store_type,0.054684
3,sales_region,0.053184


In [725]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [726]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 445807.44754956476
Mean squared error for test data: 533817.2260452982


In [727]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 8965767.83761023 12745873.63445806 12877806.92577475]


In [728]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8965767.837610226 12877806.925774746 11529816.132614344


# Stacking Regressor

In [729]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [730]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [731]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 667879.7666077738
Mean squared error for test data: 924916.6961522154


In [732]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [14443021.91634159 17023112.45280699 14414142.12818556]


In [733]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

14414142.128185557 17023112.452806987 15293425.499111377


In [734]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [735]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [736]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 151894.040686895
Mean squared error for test data: 543985.6344087062


In [737]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 8567956.71762057 16040418.61423082 14152697.25359689]


In [738]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8567956.717620572 16040418.614230815 12920357.528482758


# select 21 columns based on sfs for xgb regressor (43 columns)

In [754]:
#Taking columns that are needed
df_new=df1[['recyclable_package', 'low_fat', 'department', 'family', 'promotion_name', 'media_type', 'sales_region', 'store_type', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist', 'state_province', 'country', 'marital_status', 'yearly_income', 'gender', 'education','member_card', 'occupation', 'houseowner','store_sales', 'store_cost', 'unit_sales', 'SRP', 'gross_weight', 'net_weight', 'units_per_case', 'cases_per_pallet', 'shelf_width', 'shelf_height', 'shelf_depth', 'cost', 'store_sqft', 'grocery_sqft', 'frozen_sqft', 'meat_sqft', 'total_children', 'num_children_at_home', 'num_cars_owned', 'promotion_period']]

In [755]:
df_new["department"] = df_new["department"].apply(func_dept)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\506778296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["department"] = df_new["department"].apply(func_dept)


In [756]:
df_new['promotion_name']=df_new['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\2408714610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['promotion_name']=df_new['promotion_name'].apply(func_prom_name)


In [757]:
le=LabelEncoder()
df_new_num=df_new.select_dtypes(include=np.number)
df_new_cat=df_new.select_dtypes(exclude=np.number)

In [758]:
df_new_num

,recyclable_package,low_fat,coffee_bar,video_store,salad_bar,prepared_food,florist,store_sales,store_cost,unit_sales,...,shelf_depth,cost,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,total_children,num_children_at_home,num_cars_owned,promotion_period
0,1,0,0,0,1,1,0,7.12,2.2784,4,...,19.90,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
1,0,1,0,0,1,1,0,5.76,1.7856,4,...,11.00,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
2,1,0,0,0,1,1,0,5.55,2.0535,3,...,5.17,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
3,0,1,0,0,1,1,0,7.48,2.2440,2,...,10.40,12950,39696.0,24390.0,9184.0,6122.0,2,0,2,4
4,1,0,0,0,1,1,0,2.28,1.0488,4,...,8.98,12950,39696.0,24390.0,9184.0,6122.0,2,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,0,1,1,0,0,0,1,7.68,3.5328,3,...,20.70,13006,34452.0,27463.0,4193.0,2795.0,4,0,3,3
70747,1,0,1,0,0,0,1,2.85,0.8835,5,...,4.14,13006,34452.0,27463.0,4193.0,2795.0,4,0,3,3
70748,0,0,1,0,0,0,1,4.29,1.4586,3,...,21.10,13006,34452.0,27463.0,4193.0,2795.0,1,0,2,3
70749,1,1,1,0,0,0,1,5.18,2.0720,2,...,16.10,13006,34452.0,27463.0,4193.0,2795.0,1,0,2,3


In [759]:
df_new_cat

,department,family,promotion_name,media_type,sales_region,store_type,state_province,country,marital_status,yearly_income,gender,education,member_card,occupation,houseowner
0,Household,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
1,Dairy,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
2,Health and Hygiene,Non-Consumable,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,M,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
3,Produce,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,S,$50K - $70K,M,High School Degree,Normal,Skilled Manual,Y
4,Ready_to_eat,Food,Sale_discount,Cash Register Handout,North West,Supermarket,WA,USA,S,$50K - $70K,M,High School Degree,Normal,Skilled Manual,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,Canada,S,$110K - $130K,M,High School Degree,Silver,Management,Y
70747,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,Canada,S,$110K - $130K,M,High School Degree,Silver,Management,Y
70748,Household,Non-Consumable,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,Canada,S,$90K - $110K,M,Bachelors Degree,Golden,Professional,Y
70749,Produce,Food,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,BC,Canada,S,$90K - $110K,M,Bachelors Degree,Golden,Professional,Y


In [761]:
for col in df_new_cat.columns:
    df_new_cat[col] = le.fit_transform(df_new_cat[col])

In [762]:
df_new=pd.concat([df_new_cat,df_new_num],axis=1)

In [763]:
df_new

,department,family,promotion_name,media_type,sales_region,store_type,state_province,country,marital_status,yearly_income,...,shelf_depth,cost,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,total_children,num_children_at_home,num_cars_owned,promotion_period
0,7,2,2,1,5,4,9,2,0,4,...,19.90,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
1,4,1,2,1,5,4,9,2,0,4,...,11.00,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
2,6,2,2,1,5,4,9,2,0,4,...,5.17,12950,39696.0,24390.0,9184.0,6122.0,2,2,4,4
3,10,1,2,1,5,4,9,2,1,5,...,10.40,12950,39696.0,24390.0,9184.0,6122.0,2,0,2,4
4,11,1,2,1,5,4,9,2,1,5,...,8.98,12950,39696.0,24390.0,9184.0,6122.0,2,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,10,1,2,5,0,2,0,0,1,1,...,20.70,13006,34452.0,27463.0,4193.0,2795.0,4,0,3,3
70747,10,1,2,5,0,2,0,0,1,1,...,4.14,13006,34452.0,27463.0,4193.0,2795.0,4,0,3,3
70748,7,2,2,5,0,2,0,0,1,7,...,21.10,13006,34452.0,27463.0,4193.0,2795.0,1,0,2,3
70749,10,1,2,5,0,2,0,0,1,7,...,16.10,13006,34452.0,27463.0,4193.0,2795.0,1,0,2,3


In [764]:
#Setting target variable
X = df_new.drop(columns=['cost'])
y = df_new['cost']

In [765]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [766]:
xgbr=XGBRegressor(random_state=42)
model_sfs=sfs(estimator=xgbr,k_features=20,forward=False,scoring="neg_mean_squared_error",cv=3,n_jobs=-1)
model_sfs.fit(X_train,y_train)
model_sfs.k_feature_names_

('promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'yearly_income',
 'gender',
 'education',
 'member_card',
 'occupation',
 'houseowner',
 'coffee_bar',
 'salad_bar',
 'prepared_food',
 'store_sqft',
 'grocery_sqft',
 'frozen_sqft',
 'total_children',
 'num_children_at_home',
 'num_cars_owned',
 'promotion_period')

In [767]:
df_final_xgb=df1[['promotion_name',
 'media_type',
 'sales_region',
 'store_type',
 'yearly_income',
 'gender',
 'education',
 'member_card',
 'occupation',
 'houseowner',
 'coffee_bar',
 'salad_bar',
 'prepared_food',
 'store_sqft',
 'grocery_sqft',
 'frozen_sqft',
 'total_children',
 'num_children_at_home',
 'num_cars_owned',
 'promotion_period',"cost"]]

In [770]:
df_final_xgb['promotion_name']=df_final_xgb['promotion_name'].apply(func_prom_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_720\1081754899.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_xgb['promotion_name']=df_final_xgb['promotion_name'].apply(func_prom_name)


In [771]:
le=LabelEncoder()
df_final_xgb_num=df_final_xgb.select_dtypes(include=np.number)
df_final_xgb_cat=df_final_xgb.select_dtypes(exclude=np.number)

In [772]:
df_final_xgb_num

,coffee_bar,salad_bar,prepared_food,store_sqft,grocery_sqft,frozen_sqft,total_children,num_children_at_home,num_cars_owned,promotion_period,cost
0,0,1,1,39696.0,24390.0,9184.0,2,2,4,4,12950
1,0,1,1,39696.0,24390.0,9184.0,2,2,4,4,12950
2,0,1,1,39696.0,24390.0,9184.0,2,2,4,4,12950
3,0,1,1,39696.0,24390.0,9184.0,2,0,2,4,12950
4,0,1,1,39696.0,24390.0,9184.0,2,0,2,4,12950
...,...,...,...,...,...,...,...,...,...,...,...
70746,1,0,0,34452.0,27463.0,4193.0,4,0,3,3,13006
70747,1,0,0,34452.0,27463.0,4193.0,4,0,3,3,13006
70748,1,0,0,34452.0,27463.0,4193.0,1,0,2,3,13006
70749,1,0,0,34452.0,27463.0,4193.0,1,0,2,3,13006


In [773]:
df_final_xgb_cat

,promotion_name,media_type,sales_region,store_type,yearly_income,gender,education,member_card,occupation,houseowner
0,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
1,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
2,Sale_discount,Cash Register Handout,North West,Supermarket,$30K - $50K,F,High School Degree,Bronze,Skilled Manual,Y
3,Sale_discount,Cash Register Handout,North West,Supermarket,$50K - $70K,M,High School Degree,Normal,Skilled Manual,Y
4,Sale_discount,Cash Register Handout,North West,Supermarket,$50K - $70K,M,High School Degree,Normal,Skilled Manual,Y
...,...,...,...,...,...,...,...,...,...,...
70746,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$110K - $130K,M,High School Degree,Silver,Management,Y
70747,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$110K - $130K,M,High School Degree,Silver,Management,Y
70748,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$90K - $110K,M,Bachelors Degree,Golden,Professional,Y
70749,Sale_discount,In-Store Coupon,Canada West,Mid-Size Grocery,$90K - $110K,M,Bachelors Degree,Golden,Professional,Y


In [774]:
for col in df_final_xgb_cat.columns:
    df_final_xgb_cat[col] = le.fit_transform(df_final_xgb_cat[col])

In [775]:
df_final_xgb_cat

,promotion_name,media_type,sales_region,store_type,yearly_income,gender,education,member_card,occupation,houseowner
0,2,1,5,4,4,0,2,0,4,1
1,2,1,5,4,4,0,2,0,4,1
2,2,1,5,4,4,0,2,0,4,1
3,2,1,5,4,5,1,2,2,4,1
4,2,1,5,4,5,1,2,2,4,1
...,...,...,...,...,...,...,...,...,...,...
70746,2,5,0,2,1,1,2,3,1,1
70747,2,5,0,2,1,1,2,3,1,1
70748,2,5,0,2,7,1,0,1,3,1
70749,2,5,0,2,7,1,0,1,3,1


In [776]:
df_final_xgb=pd.concat([df_final_xgb_cat,df_final_xgb_num],axis=1)

In [777]:
df_final_xgb

,promotion_name,media_type,sales_region,store_type,yearly_income,gender,education,member_card,occupation,houseowner,...,salad_bar,prepared_food,store_sqft,grocery_sqft,frozen_sqft,total_children,num_children_at_home,num_cars_owned,promotion_period,cost
0,2,1,5,4,4,0,2,0,4,1,...,1,1,39696.0,24390.0,9184.0,2,2,4,4,12950
1,2,1,5,4,4,0,2,0,4,1,...,1,1,39696.0,24390.0,9184.0,2,2,4,4,12950
2,2,1,5,4,4,0,2,0,4,1,...,1,1,39696.0,24390.0,9184.0,2,2,4,4,12950
3,2,1,5,4,5,1,2,2,4,1,...,1,1,39696.0,24390.0,9184.0,2,0,2,4,12950
4,2,1,5,4,5,1,2,2,4,1,...,1,1,39696.0,24390.0,9184.0,2,0,2,4,12950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,2,5,0,2,1,1,2,3,1,1,...,0,0,34452.0,27463.0,4193.0,4,0,3,3,13006
70747,2,5,0,2,1,1,2,3,1,1,...,0,0,34452.0,27463.0,4193.0,4,0,3,3,13006
70748,2,5,0,2,7,1,0,1,3,1,...,0,0,34452.0,27463.0,4193.0,1,0,2,3,13006
70749,2,5,0,2,7,1,0,1,3,1,...,0,0,34452.0,27463.0,4193.0,1,0,2,3,13006


In [778]:
#Setting target variable
X = df_final_xgb.drop(columns=['cost'])
y = df_final_xgb['cost']

In [779]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression

In [780]:
model_lr=sma.OLS(y_train,sma.add_constant(X_train)).fit()

In [781]:
model_lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     87.08
Date:                Mon, 26 Feb 2024   Prob (F-statistic):               0.00
Time:                        03:13:51   Log-Likelihood:            -5.3205e+05
No. Observations:               56600   AIC:                         1.064e+06
Df Residuals:                   56579   BIC:                         1.064e+06
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.165e+04    147.022     79.247      0.000    1.14e+04    1.19e+04
promotion_name         -86.2801     12.928     -6.674      0.000    -111.619     -60.941
media_type             -23.3239      3.600     -6.479      0.000     -30.380     -16.268
sales_region          -161.2389      9.206    -17.514      0.000    -179.283    -143.195
store_type              39.3275     14.814      2.655      0.008      10.292      68.363
yearly_income          -10.4786      7.962     -1.316      0.188     -26.085       5.128
gender                 -28.8917     24.702     -1.170      0.242     -77.307      19.524
education              -10.5561      9.876     -1.069      0.285     -29.913       8.801
member_card              3.8794     13.664      0.284      0.776     -22.901      30.660
occupation              -0.4703     11.731     -0.040      0.968     -23.464      22.523
houseowner              35.8306     25.902      1.383      0.167     -14.937      86.598
coffee_bar            -106.7288     48.721     -2.191      0.028    -202.222     -11.236
salad_bar             -966.3410     89.541    -10.792      0.000   -1141.841    -790.841
prepared_food          589.8359     98.094      6.013      0.000     397.570     782.102
store_sqft               0.8415      0.084     10.073      0.000       0.678       1.005
grocery_sqft            -0.8668      0.083    -10.448      0.000      -1.029      -0.704
frozen_sqft             -1.3695      0.140     -9.779      0.000      -1.644      -1.095
total_children         -18.3975      9.001     -2.044      0.041     -36.039      -0.756
num_children_at_home   -28.2039     10.723     -2.630      0.009     -49.220      -7.188
num_cars_owned          42.7953     12.979      3.297      0.001      17.357      68.234
promotion_period       -73.2018      3.238    -22.605      0.000     -79.549     -66.855
==============================================================================
Omnibus:                    37826.886   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3249.320
Skew:                           0.062   Prob(JB):                         0.00
Kurtosis:                       1.833   Cond. No.                     4.94e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.94e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [782]:
significant_pvalues = model_lr.pvalues[model_lr.pvalues < 0.05]

# Print the significant p-values
if not significant_pvalues.empty:
    print("Significant p-values:")
    print(significant_pvalues)
else:
    print("No significant p-values.")

Significant p-values:
const                    0.000000e+00
promotion_name           2.513939e-11
media_type               9.336119e-11
sales_region             1.692367e-68
store_type               7.939509e-03
coffee_bar               2.848323e-02
salad_bar                3.982148e-27
prepared_food            1.833045e-09
store_sqft               7.650335e-24
grocery_sqft             1.578912e-25
frozen_sqft              1.442536e-22
total_children           4.096351e-02
num_children_at_home     8.532456e-03
num_cars_owned           9.768134e-04
promotion_period        1.211818e-112
dtype: float64


In [783]:
ypred_train=model_lr.predict(sma.add_constant(X_train))
ypred_test=model_lr.predict(sma.add_constant(X_test))

In [784]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8559177.232151356
Mean squared error for test data: 8564874.3697949


In [785]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [9241852.45057392 9440253.17577214 8677627.8120915 ]


In [786]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8677627.8120915 9440253.175772138 9119911.146145852


# Decision Tree Regressor

In [787]:
dtr=DecisionTreeRegressor(random_state=42)
model_dtr=dtr.fit(X_train,y_train)

In [788]:
df_imp_dt=pd.DataFrame({"feature":X_train.columns,"importance":model_dtr.feature_importances_})
df_imp_dt.sort_values(by="importance",ascending=False)

,feature,importance
1,media_type,0.305166
19,promotion_period,0.160833
0,promotion_name,0.141657
15,frozen_sqft,0.105135
14,grocery_sqft,0.049542
13,store_sqft,0.047017
2,sales_region,0.036691
3,store_type,0.035339
10,coffee_bar,0.034994
12,prepared_food,0.018853


In [789]:
ypred_train=model_dtr.predict(X_train)
ypred_test=model_dtr.predict(X_test)

In [790]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 56731.464137245464
Mean squared error for test data: 115803.47352630264


In [791]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(DecisionTreeRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [13101658.60425308 13461703.01468282 12647286.14081187]


In [792]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

12647286.140811868 13461703.014682822 13070215.919915922


# Random Forest Regressor

In [793]:
rfr=RandomForestRegressor(random_state=42,n_jobs=-1)
model_rfr=rfr.fit(X_train,y_train)

In [794]:
df_imp_rf=pd.DataFrame({"feature":X_train.columns,"importance":model_rfr.feature_importances_})
df_imp_rf.sort_values(by="importance",ascending=False)

,feature,importance
1,media_type,0.307053
19,promotion_period,0.201641
15,frozen_sqft,0.103566
0,promotion_name,0.098402
14,grocery_sqft,0.066517
13,store_sqft,0.058903
2,sales_region,0.040276
3,store_type,0.039230
10,coffee_bar,0.016254
11,salad_bar,0.009998


In [795]:
ypred_train=model_rfr.predict(X_train)
ypred_test=model_rfr.predict(X_test)

In [796]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 60467.87342717711
Mean squared error for test data: 121192.39580749674


In [797]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(RandomForestRegressor(random_state=42,n_jobs=-1), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10600186.67082096 12128953.18154587 11699893.22164118]


In [798]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10600186.670820955 12128953.181545869 11476344.358002668


# Ada Boost Regressor

In [799]:
abr=AdaBoostRegressor(random_state=42)
model_abr=abr.fit(X_train,y_train)

In [800]:
df_imp_abr=pd.DataFrame({"feature":X_train.columns,"importance":model_abr.feature_importances_})
df_imp_abr.sort_values(by="importance",ascending=False)

,feature,importance
1,media_type,0.407097
19,promotion_period,0.205162
15,frozen_sqft,0.142971
14,grocery_sqft,0.103893
2,sales_region,0.078240
13,store_sqft,0.037111
3,store_type,0.021372
0,promotion_name,0.004155
8,occupation,0.000000
9,houseowner,0.000000


In [801]:
ypred_train=model_abr.predict(X_train)
ypred_test=model_abr.predict(X_test)

In [802]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 8024895.706718391
Mean squared error for test data: 8045030.39577812


In [803]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(AdaBoostRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [8733995.90526298 9901703.87948462 9048254.26440603]


In [804]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

8733995.905262984 9901703.879484622 9227984.683051214


# Gradient Boost Regressor

In [805]:
gbr=GradientBoostingRegressor(random_state=42)
model_gbr=gbr.fit(X_train,y_train)

In [806]:
df_imp_gbr=pd.DataFrame({"feature":X_train.columns,"importance":model_gbr.feature_importances_})
df_imp_gbr.sort_values(by="importance",ascending=False)

,feature,importance
1,media_type,0.282823
19,promotion_period,0.157516
14,grocery_sqft,0.133322
15,frozen_sqft,0.119635
0,promotion_name,0.104435
13,store_sqft,0.064623
3,store_type,0.054283
2,sales_region,0.033539
10,coffee_bar,0.022791
12,prepared_food,0.013993


In [807]:
ypred_train=model_gbr.predict(X_train)
ypred_test=model_gbr.predict(X_test)

In [808]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 5180180.221710047
Mean squared error for test data: 5232807.900862917


In [809]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(GradientBoostingRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [ 9887113.91159117 11320622.93316286 11223065.82680246]


In [810]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

9887113.911591174 11320622.933162864 10810267.5571855


# XGB Regressor

In [811]:
xgbr=XGBRegressor(random_state=42)
model_xgbr=xgbr.fit(X_train,y_train)

In [812]:
df_imp_xgbr=pd.DataFrame({"feature":X_train.columns,"importance":model_xgbr.feature_importances_})
df_imp_xgbr.sort_values(by="importance",ascending=False)

,feature,importance
19,promotion_period,0.128206
15,frozen_sqft,0.124040
14,grocery_sqft,0.122034
0,promotion_name,0.119356
13,store_sqft,0.119002
1,media_type,0.084422
2,sales_region,0.079648
3,store_type,0.058940
11,salad_bar,0.048520
10,coffee_bar,0.046401


In [813]:
ypred_train=model_xgbr.predict(X_train)
ypred_test=model_xgbr.predict(X_test)

In [814]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 408097.1603184813
Mean squared error for test data: 459943.5336801592


In [815]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(XGBRegressor(random_state=42), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [10483470.01910477 10914187.86685486 11499364.12726237]


In [816]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10483470.01910477 11499364.127262374 10965674.004407333


# Stacking Regressor

In [817]:
base_estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=DecisionTreeRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [818]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [819]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 109000.54988065182
Mean squared error for test data: 189875.38571485857


In [820]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("XGBoostRegressor", XGBRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=DecisionTreeRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [13670059.120511   15417424.57462686 11571889.47135649]


In [821]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

11571889.471356485 15417424.574626865 13553124.388831452


In [822]:
base_estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
]
sr=StackingRegressor(estimators=base_estimators,final_estimator=XGBRegressor(random_state=42))
model_sr=sr.fit(X_train,y_train)

In [823]:
ypred_train=model_sr.predict(X_train)
ypred_test=model_sr.predict(X_test)

In [824]:
mse_train=mean_squared_error(y_train,ypred_train)
mse_test=mean_squared_error(y_test,ypred_test)
print("Mean squared error for train data:",mse_train)
print("Mean squared error for test data:",mse_test)

Mean squared error for train data: 64195.57182884686
Mean squared error for test data: 125715.22844858105


In [825]:
# Example: Cross-validation with 3 folds
scores = cross_val_score(StackingRegressor(estimators=[
    ("DecisionTreeRegressor", DecisionTreeRegressor(random_state=42)),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42, n_jobs=-1))
],final_estimator=XGBRegressor(random_state=42)), X, y, cv=3, scoring='neg_mean_squared_error')
print("Cross-validated RMSE:", -scores)

Cross-validated RMSE: [11011734.76500752 13467954.23350217 10208304.26392822]


In [826]:
print(np.min(-scores),np.max(-scores),np.mean(-scores))

10208304.26392822 13467954.23350217 11562664.420812637
